In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from scipy.signal import savgol_filter
from geopy.distance import geodesic
import warnings
import logging
from datetime import datetime
import json
import os

# 配置日志和常量
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
warnings.filterwarnings('ignore')

# 系统常量 - 修正为微秒
GRAVITY = 9.81
MAX_TIME_DIFF = 5000000  # 最大时间差（微秒）5秒
MAX_SAMPLES = 100000     # 增加最大样本数
VERSION = "2.1"

class EcoDrivingAnalyzer:
    """生态驾驶分析主类 - 微秒时间戳版本"""
    
    def __init__(self, config=None):
        """初始化分析器"""
        self.config = config or self._default_config()
        self.raw_data = {}
        self.processed_data = None
        self.features = {}
        self.results = {}
        self.time_offset = None  # 用于计算相对时间
        
        logger.info(f"EcoDrivingAnalyzer v{VERSION} initialized (microsecond timestamps)")
    
    def _default_config(self):
        """默认配置"""
        return {
            'hard_brake_threshold': -2.5,
            'speed_drop_threshold': 8,
            'min_brake_speed': 15,
            'energy_model': {
                'vehicle_mass': 1800,          # 更轻的乘用车
                'drag_coefficient': 0.28,      # 现代轿车风阻系数
                'frontal_area': 2.3,           # 现代轿车迎风面积
                'rolling_resistance': 0.012    # 现代轮胎滚动阻力
            },
            'clustering': {
                'n_clusters': 3,
                'features': ['speed', 'acceleration', 'jerk', 'turning_rate']
            }
        }
    
    def parse_sensor_data(self, file_path):
        """解析传感器数据文件 - 修正微秒时间戳处理"""
        logger.info(f"Parsing sensor data from {file_path}")
        
        gps_rows, acc_rows, gyro_rows, rot_rows = [], [], [], []
        total_rows = 0
        
        try:
            with open(file_path, "r") as file:
                for line_num, line in enumerate(file):
                    values = line.strip().split(",")
                    if len(values) < 2:
                        continue
                    
                    try:
                        data_type = int(values[0])
                    except ValueError:
                        continue
                    
                    # 修正数据解析逻辑
                    if data_type == 0 and len(values) >= 6:  # GPS数据
                        gps_rows.append(values)
                    elif data_type == 1 and len(values) >= 5:  # 加速度计数据
                        acc_rows.append(values)
                    elif data_type == 2 and len(values) >= 5:  # 陀螺仪数据
                        gyro_rows.append(values)
                    elif data_type == 3 and len(values) >= 6:  # 四元数数据
                        rot_rows.append(values)
                    
                    total_rows += 1
                    if total_rows >= MAX_SAMPLES:
                        break
            
            logger.info(f"Parsed {total_rows} lines from {file_path}")
        
        except Exception as e:
            logger.error(f"Error reading file: {e}")
            return False
        
        # 处理GPS数据
        if gps_rows:
            print(f"\n=== GPS数据解析 ===")
            print(f"GPS原始行数: {len(gps_rows)}")
            print(f"样本数据: {gps_rows[0]}")
            
            # 处理GPS数据格式：0,timestamp,lat,lon,alt,speed_kmh,satellites
            gps_data = []
            for row in gps_rows:
                try:
                    if len(row) >= 6:
                        gps_data.append([
                            int(row[0]),      # data_type
                            int(row[1]),      # timestamp (微秒)
                            float(row[2]),    # latitude
                            float(row[3]),    # longitude
                            float(row[4]),    # altitude
                            float(row[5]),    # speed_kmh
                            int(row[6]) if len(row) > 6 else 12  # satellites
                        ])
                except (ValueError, IndexError) as e:
                    continue
            
            if gps_data:
                gps_df = pd.DataFrame(gps_data, columns=[
                    "data_type", "timestamp", "latitude", "longitude", 
                    "altitude", "speed_kmh", "satellites"
                ])
                
                gps_df["speed_ms"] = gps_df["speed_kmh"] / 3.6
                
                # 设置时间偏移量
                if self.time_offset is None:
                    self.time_offset = gps_df['timestamp'].min()
                
                # 计算相对时间（秒）
                gps_df['relative_time_s'] = (gps_df['timestamp'] - self.time_offset) / 1000000.0
                
                print(f"GPS数据质量检查:")
                print(f"  数据点数: {len(gps_df)}")
                print(f"  时间跨度: {gps_df['relative_time_s'].max():.1f} 秒 ({gps_df['relative_time_s'].max()/60:.1f} 分钟)")
                print(f"  速度范围: {gps_df['speed_kmh'].min():.1f} - {gps_df['speed_kmh'].max():.1f} km/h")
                print(f"  位置变化: 纬度 {gps_df['latitude'].max()-gps_df['latitude'].min():.6f}°, 经度 {gps_df['longitude'].max()-gps_df['longitude'].min():.6f}°")
                
                # 数据清洗
                gps_df = gps_df[gps_df["speed_kmh"] <= 200]
                gps_df = gps_df[gps_df["latitude"].between(-90, 90)]
                gps_df = gps_df[gps_df["longitude"].between(-180, 180)]
                
                self.raw_data['gps'] = gps_df
        
        # 处理加速度传感器数据
        if acc_rows:
            print(f"\n=== 加速度传感器数据解析 ===")
            print(f"加速度原始行数: {len(acc_rows)}")
            print(f"样本数据: {acc_rows[0]}")
            
            # 处理加速度数据格式：1,timestamp,timestamp_us,x,y,z
            acc_data = []
            for row in acc_rows:
                try:
                    if len(row) >= 5:
                        acc_data.append([
                            int(row[0]),      # data_type
                            int(row[1]),      # timestamp (微秒)
                            int(row[2]),      # timestamp_us (额外的微秒)
                            float(row[3]),    # x
                            float(row[4]),    # y
                            float(row[5]) if len(row) > 5 else 0.0  # z
                        ])
                except (ValueError, IndexError):
                    continue
            
            if acc_data:
                acc_df = pd.DataFrame(acc_data, columns=[
                    "data_type", "timestamp", "timestamp_us", "x", "y", "z"
                ])
                
                # 计算完整时间戳（主时间戳 + 微秒偏移）
                acc_df['full_timestamp'] = acc_df['timestamp'] + acc_df['timestamp_us']
                
                if self.time_offset is None:
                    self.time_offset = acc_df['full_timestamp'].min()
                
                acc_df['relative_time_s'] = (acc_df['full_timestamp'] - self.time_offset) / 1000000.0
                
                print(f"加速度数据质量检查:")
                print(f"  数据点数: {len(acc_df)}")
                print(f"  时间跨度: {acc_df['relative_time_s'].max():.1f} 秒")
                
                # 检查加速度范围和缩放
                max_acc = max(acc_df['x'].abs().max(), acc_df['y'].abs().max(), acc_df['z'].abs().max())
                print(f"  原始加速度最大值: {max_acc:.2f}")
                
                # 智能缩放 - 根据数据范围判断单位
                if max_acc > 50:  # 如果数值很大，可能需要缩放
                    if max_acc > 1000:
                        scale_factor = 1000.0
                        print("  检测到加速度单位可能是 mg，应用1000x缩放")
                    else:
                        scale_factor = 10.0
                        print("  检测到加速度单位异常，应用10x缩放")
                    
                    acc_df[["x", "y", "z"]] = acc_df[["x", "y", "z"]] / scale_factor
                    print(f"  缩放后范围: {acc_df['x'].min():.2f} - {acc_df['x'].max():.2f}")
                
                acc_df["magnitude"] = np.sqrt(acc_df["x"]**2 + acc_df["y"]**2 + acc_df["z"]**2)
                self.raw_data['acc'] = acc_df
        
        # 处理陀螺仪数据
        if gyro_rows:
            print(f"\n=== 陀螺仪数据解析 ===")
            gyro_data = []
            for row in gyro_rows:
                try:
                    if len(row) >= 5:
                        gyro_data.append([
                            int(row[0]),      # data_type
                            int(row[1]),      # timestamp
                            int(row[2]),      # timestamp_us
                            float(row[3]),    # x
                            float(row[4]),    # y
                            float(row[5]) if len(row) > 5 else 0.0  # z
                        ])
                except (ValueError, IndexError):
                    continue
            
            if gyro_data:
                gyro_df = pd.DataFrame(gyro_data, columns=[
                    "data_type", "timestamp", "timestamp_us", "x", "y", "z"
                ])
                gyro_df['full_timestamp'] = gyro_df['timestamp'] + gyro_df['timestamp_us']
                
                if self.time_offset is None:
                    self.time_offset = gyro_df['full_timestamp'].min()
                
                gyro_df['relative_time_s'] = (gyro_df['full_timestamp'] - self.time_offset) / 1000000.0
                gyro_df["turning_rate"] = gyro_df["z"].abs()
                
                print(f"  数据点数: {len(gyro_df)}")
                print(f"  Z轴角速度范围: {gyro_df['z'].min():.3f} - {gyro_df['z'].max():.3f} rad/s")
                
                self.raw_data['gyro'] = gyro_df
        
        return True
    
    def extract_driving_features(self):
        """提取驾驶特征 - 修正时间处理和Jerk计算"""
        logger.info("Extracting driving features with microsecond precision")
        
        # GPS路线特征
        if 'gps' in self.raw_data and not self.raw_data['gps'].empty:
            gps_df = self.raw_data['gps'].copy().sort_values('timestamp').reset_index(drop=True)
            
            print(f"\n=== GPS特征提取 ===")
            
            # 计算距离和坡度
            distances, gradients = [], []
            for i in range(1, len(gps_df)):
                p1 = (gps_df["latitude"].iloc[i-1], gps_df["longitude"].iloc[i-1])
                p2 = (gps_df["latitude"].iloc[i], gps_df["longitude"].iloc[i])
                try:
                    d = geodesic(p1, p2).meters
                except Exception:
                    d = 0
                distances.append(d)
                
                alt_diff = gps_df["altitude"].iloc[i] - gps_df["altitude"].iloc[i-1]
                gradient = alt_diff / d if d > 0.5 else 0  # 距离>0.5米才计算坡度
                gradients.append(gradient)
            
            if distances:
                gps_df.loc[1:, "distance"] = distances
                gps_df.loc[1:, "gradient"] = gradients
            gps_df["distance"] = gps_df.get("distance", 0).fillna(0)
            gps_df["gradient"] = gps_df.get("gradient", 0).fillna(0)
            
            # 修正的加速度计算 - 使用微秒精度，放宽限制
            dt_values = gps_df["timestamp"].diff() / 1000000.0  # 转换为秒
            speed_changes_ms = gps_df["speed_ms"].diff()
            
            accelerations = []
            for i in range(len(gps_df)):
                if i == 0:
                    accelerations.append(0.0)
                else:
                    dt = dt_values.iloc[i]
                    speed_change = speed_changes_ms.iloc[i]
                    
                    # 修正时间范围检查，放宽加速度限制
                    if 0.01 < dt < 300:  # 10ms 到 5分钟
                        acc = speed_change / dt
                        acc = max(-15, min(15, acc))  # 放宽加速度范围到±15 m/s²
                        accelerations.append(acc)
                    else:
                        # 对于异常时间间隔，使用前一个加速度的衰减值
                        if i > 1:
                            accelerations.append(accelerations[-1] * 0.8)
                        else:
                            accelerations.append(0.0)
            
            gps_df["acceleration"] = accelerations
            
            # 检查GPS加速度计算结果
            acc_nonzero = (np.array(accelerations) != 0).sum()
            acc_range = [min(accelerations), max(accelerations)]
            
            print(f"GPS加速度计算结果:")
            print(f"  非零值: {acc_nonzero}/{len(accelerations)} ({acc_nonzero/len(accelerations)*100:.1f}%)")
            print(f"  范围: {acc_range[0]:.3f} - {acc_range[1]:.3f} m/s²")
            print(f"  平均: {np.mean(accelerations):.3f} m/s²")
            print(f"  标准差: {np.std(accelerations):.3f} m/s²")
            
            # 如果GPS加速度仍然问题很大，使用改进的合成方法
            if abs(max(accelerations)) < 0.01 or acc_nonzero < len(accelerations) * 0.1:
                print("⚠️  GPS加速度计算异常，使用改进的合成方法...")
                
                # 使用速度的变化趋势生成更真实的加速度
                speed_smooth = gps_df['speed_ms'].rolling(window=3, center=True).mean().fillna(gps_df['speed_ms'])
                speed_diff = speed_smooth.diff()
                time_diff = gps_df['relative_time_s'].diff()
                
                synthetic_acc = []
                for i in range(len(gps_df)):
                    if i == 0:
                        synthetic_acc.append(0.0)
                    else:
                        dt = time_diff.iloc[i]
                        if dt > 0:
                            acc = speed_diff.iloc[i] / dt
                            acc = max(-12, min(12, acc))  # 合成加速度限制稍微严格一些
                            synthetic_acc.append(acc)
                        else:
                            synthetic_acc.append(0.0)
                
                gps_df['acceleration'] = synthetic_acc
                print(f"  合成加速度范围: {min(synthetic_acc):.3f} - {max(synthetic_acc):.3f} m/s²")
            
            self.features['route'] = gps_df
        
        # 加速度传感器特征 - 修正时间处理和Jerk计算
        if 'acc' in self.raw_data and not self.raw_data['acc'].empty:
            acc_df = self.raw_data['acc'].copy().sort_values('full_timestamp').reset_index(drop=True)
            
            print(f"\n=== 加速度传感器特征提取 ===")
            
            # 重新计算magnitude，确保单位正确
            print(f"原始加速度数据统计:")
            print(f"  X: {acc_df['x'].min():.3f} - {acc_df['x'].max():.3f}")
            print(f"  Y: {acc_df['y'].min():.3f} - {acc_df['y'].max():.3f}")
            print(f"  Z: {acc_df['z'].min():.3f} - {acc_df['z'].max():.3f}")
            
            # 重新计算magnitude
            acc_df["magnitude"] = np.sqrt(acc_df["x"]**2 + acc_df["y"]**2 + acc_df["z"]**2)
            
            # 计算Jerk（急动度）- 修正版本
            dt_acc = acc_df["full_timestamp"].diff() / 1000000.0  # 转换为秒
            magnitude_changes = acc_df["magnitude"].diff()

            # 调试：检查magnitude变化
            print(f"Magnitude统计: {acc_df['magnitude'].min():.3f} - {acc_df['magnitude'].max():.3f}")
            print(f"Magnitude变化统计: {magnitude_changes.min():.6f} - {magnitude_changes.max():.6f}")
            print(f"时间间隔统计: {dt_acc.min():.6f} - {dt_acc.max():.6f} 秒")
            
            # 初始化jerks列表
            jerks = []
            
            for i in range(len(acc_df)):
                if i == 0:
                    jerks.append(0.0)
                else:
                    dt = dt_acc.iloc[i]
                    mag_change = magnitude_changes.iloc[i]
                    
                    # 修正时间范围 - 加速度计采样频率更高
                    if 0.0001 < dt < 1.0:  # 0.1ms 到 1秒，放宽时间范围
                        jerk = mag_change / dt
                        # 放宽jerk范围，允许更大的瞬时变化
                        jerk = max(-500, min(500, jerk))  # 扩大jerk范围到±500
                        jerks.append(jerk)
                    else:
                        # 对于异常时间间隔，使用0而不是跳过
                        jerks.append(0.0)
            
            acc_df["jerk"] = jerks
            
            # 改进的平滑处理 - 使用更保守的平滑
            if len(acc_df) > 5:
                # 使用更小的窗口保持瞬时特性
                win = min(5, len(acc_df)//10*2+1)  # 减少窗口大小
                if win >= 3:
                    try:
                        acc_df["jerk_smooth"] = savgol_filter(acc_df["jerk"], win, 1)
                    except:
                        # 如果平滑失败，使用简单的滑动平均
                        acc_df["jerk_smooth"] = acc_df["jerk"].rolling(window=3, center=True).mean().fillna(acc_df["jerk"])
                else:
                    acc_df["jerk_smooth"] = acc_df["jerk"]
            else:
                acc_df["jerk_smooth"] = acc_df["jerk"]
            
            # 去除极端异常值
            jerk_p99 = np.percentile(np.abs(acc_df["jerk_smooth"]), 99)
            acc_df["jerk_smooth"] = np.clip(acc_df["jerk_smooth"], -jerk_p99, jerk_p99)
            
            print(f"Jerk计算结果:")
            print(f"  原始范围: {min(jerks):.3f} - {max(jerks):.3f} m/s³")
            print(f"  平滑后范围: {acc_df['jerk_smooth'].min():.3f} - {acc_df['jerk_smooth'].max():.3f} m/s³")
            print(f"  非零值: {(np.array(jerks) != 0).sum()}/{len(jerks)}")
            print(f"  99%分位数: ±{jerk_p99:.3f} m/s³")
            
            self.features['behavior'] = acc_df
        
        # 陀螺仪转向特征 - 修正计算
        if 'gyro' in self.raw_data and not self.raw_data['gyro'].empty:
            gyro_df = self.raw_data['gyro'].copy()
            
            print(f"\n=== 陀螺仪特征提取 ===")
            print(f"陀螺仪原始数据:")
            print(f"  X: {gyro_df['x'].min():.3f} - {gyro_df['x'].max():.3f}")
            print(f"  Y: {gyro_df['y'].min():.3f} - {gyro_df['y'].max():.3f}")
            print(f"  Z: {gyro_df['z'].min():.3f} - {gyro_df['z'].max():.3f}")
            
            # 重新计算转向率 - 使用所有轴的信息
            gyro_df["turning_rate"] = np.sqrt(gyro_df["x"]**2 + gyro_df["y"]**2 + gyro_df["z"]**2)
            
            print(f"转向率统计: {gyro_df['turning_rate'].min():.3f} - {gyro_df['turning_rate'].max():.3f}")
            
            self.features['turning'] = gyro_df
        
        return True
    
    def detect_hard_brake_events(self):
        """检测急刹车事件 - 修正版"""
        logger.info("Detecting hard brake events with improved algorithm")
        
        if 'route' not in self.features:
            logger.warning("No route data available for hard brake detection")
            return pd.DataFrame()
        
        df = self.features['route'].copy()
        
        # 数据平滑
        if len(df) > 5:
            window_size = min(5, len(df) // 2 * 2 + 1)
            df['speed_smooth'] = savgol_filter(df['speed_kmh'], window_size, 2)
        else:
            df['speed_smooth'] = df['speed_kmh']
        
        # 速度变化分析 - 使用相对时间
        df['speed_change'] = df['speed_smooth'].diff()
        time_diff_s = df['relative_time_s'].diff()
        df['speed_change_rate'] = df['speed_change'] / time_diff_s
        df['speed_change_pct'] = df['speed_change'] / (df['speed_smooth'].shift(1) + 1)
        
        # 针对低速场景调整的急刹车检测条件
        conditions = [
            # 条件1: 适应低速的绝对速度下降
            (df['speed_change'] < -5) & (df['speed_smooth'] > 10),  # 降低速度阈值
            
            # 条件2: 相对速度变化
            (df['speed_change_pct'] < -0.20) & (df['speed_smooth'] > 8),  # 提高相对变化敏感度
            
            # 条件3: 速度变化率（针对起步阶段）
            (df['speed_change_rate'] < -8) & (df['speed_smooth'] > 5),  # 降低速度阈值
        ]
        
        df['hard_brake_candidate'] = np.logical_or.reduce(conditions)
        
        # 速度峰值检测 - 针对低速优化
        df['is_speed_peak'] = False
        df['hard_brake_refined'] = False
        
        # 寻找局部速度峰值（降低阈值适应低速）
        for i in range(2, len(df) - 2):
            current_speed = df['speed_smooth'].iloc[i]
            prev_speed = df['speed_smooth'].iloc[i-1]
            next_speed = df['speed_smooth'].iloc[i+1]
            
            if (current_speed > prev_speed and current_speed > next_speed and current_speed > 8):  # 降低峰值阈值
                df.loc[i, 'is_speed_peak'] = True
        
        # 在峰值后检测急刹车
        for i in range(len(df)):
            if df['is_speed_peak'].iloc[i]:
                peak_speed = df['speed_smooth'].iloc[i]
                
                for j in range(i+1, min(i+6, len(df))):
                    current_speed = df['speed_smooth'].iloc[j]
                    speed_drop = peak_speed - current_speed
                    time_span = df['relative_time_s'].iloc[j] - df['relative_time_s'].iloc[i]
                    
                    # 针对低速调整的急刹车条件
                    if speed_drop > 8 and time_span < 5:  # 降低速度下降阈值
                        df.loc[j, 'hard_brake_refined'] = True
                        break
        
        # 最终急刹车事件
        df['hard_brake_final'] = df['hard_brake_candidate'] | df['hard_brake_refined']
        
        # 去重处理
        df['hard_brake_filtered'] = False
        if df['hard_brake_final'].any():
            brake_indices = df[df['hard_brake_final']].index
            filtered_indices = [brake_indices[0]] if len(brake_indices) > 0 else []
            
            for i in range(1, len(brake_indices)):
                time_gap = df['relative_time_s'].iloc[brake_indices[i]] - df['relative_time_s'].iloc[brake_indices[i-1]]
                if time_gap > 2.0:  # 2秒间隔
                    filtered_indices.append(brake_indices[i])
            
            if filtered_indices:
                df.loc[filtered_indices, 'hard_brake_filtered'] = True
        
        hard_brake_events = df[df['hard_brake_filtered']]
        
        print(f"\n=== 急刹车检测结果 ===")
        print(f"候选事件: {df['hard_brake_candidate'].sum()}")
        print(f"精炼事件: {df['hard_brake_refined'].sum()}")
        print(f"最终检测到: {len(hard_brake_events)} 个急刹车事件")
        
        if not hard_brake_events.empty:
            print("急刹车事件详情:")
            for i, (idx, event) in enumerate(hard_brake_events.iterrows()):
                time_min = event['relative_time_s'] / 60
                print(f"  事件{i+1}: {time_min:.1f}分钟 - 速度: {event['speed_kmh']:.1f} km/h, 加速度: {event['acceleration']:.2f} m/s²")
        
        # 更新路线数据
        self.features['route'] = df
        
        return hard_brake_events
    
    def synchronize_and_model_energy(self):
        """数据同步和能耗建模 - 微秒精度版本"""
        logger.info("Synchronizing data and modeling energy consumption (microsecond precision)")
        
        if 'route' not in self.features or 'behavior' not in self.features:
            logger.warning("Missing required data for energy modeling")
            return pd.DataFrame()
        
        route_df = self.features['route']
        behavior_df = self.features['behavior']
        
        # 使用微秒时间戳进行同步
        route_times = route_df['timestamp'].values
        behavior_times = behavior_df['full_timestamp'].values
        
        print(f"\n=== 数据同步 (微秒精度) ===")
        print(f"GPS数据点: {len(route_df)}")
        print(f"加速度数据点: {len(behavior_df)}")
        
        combined_data = []
        sync_errors = 0
        perfect_matches = 0
        
        for i, row in route_df.iterrows():
            gps_time = row['timestamp']
            time_diffs = np.abs(behavior_times - gps_time)
            closest_acc_idx = np.argmin(time_diffs)
            min_time_diff = time_diffs[closest_acc_idx]
            
            if min_time_diff == 0:
                perfect_matches += 1
            elif min_time_diff > MAX_TIME_DIFF:
                sync_errors += 1
                continue
            
            entry = {
                'timestamp': gps_time,
                'relative_time_s': row['relative_time_s'],
                'speed': row['speed_ms'],
                'speed_kmh': row['speed_kmh'],
                'acceleration': row['acceleration'],
                'gradient': row['gradient'],
                'jerk': behavior_df['jerk_smooth'].iloc[closest_acc_idx],
                'sync_error_us': min_time_diff  # 微秒
            }
            
            # 添加转向数据
            if 'turning' in self.features:
                turning_times = self.features['turning']['full_timestamp'].values
                closest_turn_idx = np.argmin(np.abs(turning_times - gps_time))
                turn_time_diff = np.abs(turning_times[closest_turn_idx] - gps_time)
                
                if turn_time_diff <= MAX_TIME_DIFF:
                    entry['turning_rate'] = self.features['turning']['turning_rate'].iloc[closest_turn_idx]
                else:
                    entry['turning_rate'] = 0
            else:
                entry['turning_rate'] = 0
            
            combined_data.append(entry)
        
        if not combined_data:
            logger.warning("No synchronized data points found")
            return pd.DataFrame()
        
        combined_df = pd.DataFrame(combined_data)
        
        # 能耗计算
        combined_df['energy_factor'] = self._calculate_energy_consumption(combined_df)
        
        print(f"同步结果:")
        print(f"  完美匹配: {perfect_matches}")
        print(f"  成功同步: {len(combined_data)}")
        print(f"  同步错误: {sync_errors}")
        print(f"  平均同步误差: {combined_df['sync_error_us'].mean():.0f} 微秒")
        
        self.processed_data = combined_df
        return combined_df
    
    def _calculate_energy_consumption_fixed(self, df):
        """计算能耗 - 修正版，调整参数使结果更合理"""
        config = self.config['energy_model']
        mass = config['vehicle_mass']
        g = GRAVITY
        rho = 1.225  # 空气密度
        Cd = config['drag_coefficient']
        A = config['frontal_area']
        Cr = config['rolling_resistance']
        
        power_W = np.zeros(len(df))
        
        for i in range(len(df)):
            v = max(0.1, df['speed'].iloc[i])
            θ = np.arctan(df['gradient'].iloc[i]) if abs(df['gradient'].iloc[i]) < 1 else 0
            
            F_roll = Cr * mass * g * np.cos(θ)
            F_aero = 0.5 * rho * Cd * A * v**2
            F_grad = mass * g * np.sin(θ)
            F_acc = mass * df['acceleration'].iloc[i]
            
            F_total = F_roll + F_aero + F_grad + F_acc
            F_traction = max(F_total, 0)
            
            # 添加电机效率因子，使能耗更合理
            motor_efficiency = 0.85  # 电机效率85%
            power_W[i] = F_traction * v / motor_efficiency
        
        # 计算能耗 - 使用相对时间，添加合理性检查
        if len(df) > 1:
            dt = df['relative_time_s'].diff().fillna(1.0)
            dt = np.clip(dt, 0.01, 10.0)  # 限制时间间隔
        else:
            dt = np.array([1.0])
        
        energy_J = power_W * dt.values
        
        # 限制单个数据点的能耗，避免异常值
        energy_J = np.clip(energy_J, 0, 50000)  # 限制单点最大能耗50kJ
        
        df['energy_J'] = energy_J
        
        return power_W
    
    def cluster_driving_behaviors(self):
        """驾驶行为聚类 - 改进版"""
        logger.info("Clustering driving behaviors with improved classification")
        
        if self.processed_data is None or self.processed_data.empty:
            logger.error("No processed data available for clustering")
            return {}, pd.DataFrame(), np.array([])
        
        data = self.processed_data.copy()
        
        # 确保所需列存在
        required_cols = self.config['clustering']['features'] + ['energy_J']
        for col in required_cols:
            if col not in data.columns:
                data[col] = 0.0
        
        print(f"\n=== 驾驶行为聚类 ===")
        for col in self.config['clustering']['features']:
            if col in data.columns:
                print(f"{col}: {data[col].min():.3f} - {data[col].max():.3f} (均值: {data[col].mean():.3f})")
        
        # K-Means聚类
        feats = self.config['clustering']['features']
        X = data[feats].fillna(0)
        
        # 数据标准化
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # PCA降维
        try:
            pca = PCA(n_components=2)
            X_pca = pca.fit_transform(X_scaled)
        except Exception as e:
            logger.warning(f"PCA failed: {e}")
            X_pca = X_scaled[:, :2] if X_scaled.shape[1] >= 2 else np.column_stack([X_scaled[:, 0], X_scaled[:, 0]])
        
        n_clusters = min(self.config['clustering']['n_clusters'], max(1, len(X) - 1))
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        clusters = kmeans.fit_predict(X_scaled)
        
        data['driver_cluster'] = clusters
        
        # 生成聚类配置文件
        cluster_profiles = {}
        for cid in range(n_clusters):
            data_c = data[data['driver_cluster'] == cid]
            if data_c.empty:
                cluster_profiles[cid] = {'size': 0, 'driver_type': 'unknown'}
                continue
            
            # 计算统计指标
            if len(data_c) > 1:
                dt = data_c['relative_time_s'].diff().fillna(1.0)
                dt = np.clip(dt, 0.01, 10.0)
                total_time = dt.sum()
                total_distance = (data_c['speed'] * dt).sum() / 1000.0  # km
            else:
                total_time = 1.0
                total_distance = 0.001
            
            total_energy = data_c['energy_J'].sum() / 3600000.0  # kWh (J->kWh)
            energy_eff = total_energy / max(total_distance, 0.001) * 100  # kWh/100km
            
            profile = {
                'size': len(data_c),
                'avg_speed': data_c['speed'].mean(),
                'std_speed': data_c['speed'].std(),
                'avg_acceleration': data_c['acceleration'].mean(),
                'std_acceleration': data_c['acceleration'].std(),
                'avg_jerk': data_c['jerk'].mean(),
                'std_jerk': data_c['jerk'].std(),
                'avg_turning_rate': data_c['turning_rate'].mean(),
                'std_turning_rate': data_c['turning_rate'].std(),
                'energy_efficiency': energy_eff,
                'total_time_s': total_time,
                'total_distance_km': total_distance,
            }
            
            # 改进的驾驶类型分类 - 适应起步阶段
            abs_avg_jerk = abs(profile['avg_jerk'])
            abs_avg_acc = abs(profile['avg_acceleration'])
            avg_speed = profile['avg_speed']
            avg_turning = profile['avg_turning_rate']
            
            # 针对起步阶段的分类逻辑
            if abs_avg_jerk > 15.0 and abs_avg_acc > 2.0:
                profile['driver_type'] = 'aggressive'
            elif avg_turning > 0.15:
                profile['driver_type'] = 'cornering'
            elif avg_speed > 8.0 and abs_avg_acc > 1.0:  # 调整速度阈值
                profile['driver_type'] = 'dynamic'
            elif avg_speed < 5.0 and abs_avg_jerk < 8.0:
                profile['driver_type'] = 'cautious'
            elif abs_avg_acc < 0.5 and abs_avg_jerk < 5.0:
                profile['driver_type'] = 'efficient'
            else:
                profile['driver_type'] = 'normal'
            
            cluster_profiles[cid] = profile
        
        print(f"聚类结果:")
        for cid, profile in cluster_profiles.items():
            print(f"  Cluster {cid} ({profile['driver_type']}): {profile['size']} 样本")
            print(f"    速度: {profile['avg_speed']*3.6:.1f} km/h, 加速度: {profile['avg_acceleration']:.2f} m/s²")
            print(f"    Jerk: {profile['avg_jerk']:.2f} m/s³, 能耗: {profile['energy_efficiency']:.1f} kWh/100km")
        
        # 更新处理后的数据
        self.processed_data = data
        
        # 在 cluster_driving_behaviors() 方法的最后，return 语句之前添加：
        print(f"DEBUG: type of self.processed_data after clustering: {type(self.processed_data)}")
        print(f"DEBUG: processed_data columns: {self.processed_data.columns.tolist() if isinstance(self.processed_data, pd.DataFrame) else 'Not DataFrame'}")
        
        return cluster_profiles, X_pca, clusters
    
    def generate_comprehensive_report(self):
        """生成综合分析报告"""
        logger.info("Generating comprehensive analysis report")
        
        if self.processed_data is None:
            logger.error("No processed data available for report generation")
            return None
        
        # 基本统计
        data = self.processed_data
        total_time = data['relative_time_s'].max()  # 使用相对时间
        total_distance = (data['speed'] * data['relative_time_s'].diff().fillna(1.0)).sum() / 1000.0  # km
        avg_speed = data['speed'].mean() * 3.6  # km/h
        max_speed = data['speed_kmh'].max()
        
        # 急刹车分析
        hard_brake_events = self.detect_hard_brake_events()
        
        # 驾驶行为聚类
        cluster_profiles, X_pca, clusters = self.cluster_driving_behaviors()

        # 确保processed_data被正确更新
        if self.processed_data is not None:
            data = self.processed_data  # 使用更新后的数据
        else:
            logger.error("processed_data is None after clustering")
            return None
        
        # 生成报告
        report = {
            'metadata': {
                'analysis_date': datetime.now().isoformat(),
                'version': VERSION,
                'data_points': len(data),
                'duration_minutes': total_time / 60,
                'total_distance_km': total_distance
            },
            'driving_summary': {
                'max_speed_kmh': max_speed,
                'avg_speed_kmh': avg_speed,
                'total_hard_brakes': len(hard_brake_events),
                'hard_brakes_per_hour': len(hard_brake_events) / max(total_time / 3600, 0.1)
            },
            'cluster_analysis': cluster_profiles,
            'recommendations': self._generate_recommendations(cluster_profiles, hard_brake_events)
        }
        
        self.results = {
            'report': report,
            'data': self.processed_data.copy(),
            'hard_brake_events': hard_brake_events,
            'cluster_profiles': cluster_profiles,
            'pca_data': X_pca,
            'clusters': clusters
        }
        
        return report
    
    def _generate_recommendations(self, cluster_profiles, hard_brake_events):
        """生成驾驶建议"""
        recommendations = []
        
        # 基于急刹车的建议
        if len(hard_brake_events) > 3:  # 降低阈值适应起步阶段
            recommendations.append({
                'category': 'Safety',
                'priority': 'High',
                'message': 'Multiple hard braking events detected during startup phase. Consider smoother acceleration and anticipating traffic conditions.',
                'impact': 'Reduces brake wear and improves passenger comfort'
            })
        elif len(hard_brake_events) == 0:
            recommendations.append({
                'category': 'Positive',
                'priority': 'Low',
                'message': 'Excellent! No hard braking events detected. Very smooth driving style.',
                'impact': 'Optimal brake system preservation and passenger comfort'
            })
        
        # 基于聚类分析的建议
        for cid, profile in cluster_profiles.items():
            if profile['driver_type'] == 'aggressive':
                recommendations.append({
                    'category': 'Efficiency',
                    'priority': 'Medium',
                    'message': f'Cluster {cid} shows aggressive driving patterns. Gentler acceleration can improve efficiency by 15-20%.',
                    'impact': f'Current energy efficiency: {profile["energy_efficiency"]:.1f} kWh/100km'
                })
            elif profile['driver_type'] == 'efficient':
                recommendations.append({
                    'category': 'Positive',
                    'priority': 'Low',
                    'message': f'Cluster {cid} demonstrates efficient driving behavior. Excellent energy management!',
                    'impact': f'Outstanding energy efficiency: {profile["energy_efficiency"]:.1f} kWh/100km'
                })
            elif profile['driver_type'] == 'cautious':
                recommendations.append({
                    'category': 'Performance',
                    'priority': 'Low',
                    'message': f'Cluster {cid} shows very cautious driving. Consider slightly more dynamic acceleration when safe.',
                    'impact': 'May improve traffic flow while maintaining safety'
                })
        
        return recommendations
    
    def visualize_results(self, save_plots=True):
        if not self.results:
            logger.error("No results available for visualization")
            return
        
        # 调试：检查数据类型
        print(f"DEBUG: self.results keys: {list(self.results.keys())}")
        print(f"DEBUG: type of self.results['data']: {type(self.results['data'])}")
        print(f"DEBUG: type of self.processed_data: {type(self.processed_data)}")
        
        # 直接使用 processed_data
        data = self.processed_data
        if data is None or not isinstance(data, pd.DataFrame):
            logger.error(f"Invalid processed_data type: {type(data)}")
            return
        
        hard_brake_events = self.results['hard_brake_events']
        cluster_profiles = self.results['cluster_profiles']
        
        # 创建综合仪表板
        fig = plt.figure(figsize=(20, 16))
        
        # 1. 速度时间曲线（使用相对时间）
        ax1 = plt.subplot(3, 3, 1)
        time_minutes = data['relative_time_s'] / 60
        
        plt.plot(time_minutes, data['speed_kmh'], 'b-', alpha=0.7, linewidth=2, label='Speed (km/h)')
        
        if not hard_brake_events.empty:
            brake_times = hard_brake_events['relative_time_s'] / 60
            plt.scatter(brake_times, hard_brake_events['speed_kmh'], 
                       color='red', marker='X', s=100, zorder=5, 
                       label=f'Hard Brakes ({len(hard_brake_events)})')
            
            # 添加急刹车标注
            for i, (idx, event) in enumerate(hard_brake_events.iterrows()):
                plt.annotate(f'Brake {i+1}', 
                           xy=(event['relative_time_s']/60, event['speed_kmh']),
                           xytext=(10, 20), textcoords='offset points',
                           bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.8),
                           arrowprops=dict(arrowstyle='->', color='red'))
        
        plt.title('Speed Profile with Hard Brake Events\n(Startup Phase Analysis)')
        plt.xlabel('Time (minutes)')
        plt.ylabel('Speed (km/h)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 2. 驾驶行为聚类PCA视图
        ax2 = plt.subplot(3, 3, 2)
        scatter = plt.scatter(self.results['pca_data'][:, 0], self.results['pca_data'][:, 1], 
                             c=self.results['clusters'], cmap='viridis', alpha=0.7, s=30)
        plt.title('Driver Behavior Clusters\n(PCA Visualization)')
        plt.xlabel('PCA Component 1')
        plt.ylabel('PCA Component 2')
        plt.colorbar(scatter, label='Cluster')
        plt.grid(True, alpha=0.3)
        
        # 3. 加速度分布
        ax3 = plt.subplot(3, 3, 3)
        colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown']
        for cluster in range(len(cluster_profiles)):
            cluster_data = data[data['driver_cluster'] == cluster]
            if not cluster_data.empty and 'acceleration' in cluster_data.columns:
                plt.hist(cluster_data['acceleration'], bins=15, alpha=0.6, 
                        label=f'Cluster {cluster}', density=True, 
                        color=colors[cluster % len(colors)])
        plt.title('Acceleration Distribution by Cluster\n(Startup Phase)')
        plt.xlabel('Acceleration (m/s²)')
        plt.ylabel('Density')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 4. Jerk分布
        ax4 = plt.subplot(3, 3, 4)
        for cluster in range(len(cluster_profiles)):
            cluster_data = data[data['driver_cluster'] == cluster]
            if not cluster_data.empty and 'jerk' in cluster_data.columns:
                plt.hist(cluster_data['jerk'], bins=15, alpha=0.6, 
                        label=f'Cluster {cluster}', density=True,
                        color=colors[cluster % len(colors)])
        plt.title('Jerk Distribution by Cluster\n(Instantaneous Values)')
        plt.xlabel('Jerk (m/s³)')
        plt.ylabel('Density')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 5. 驾驶类型分布
        ax5 = plt.subplot(3, 3, 5)
        driver_types = [cluster_profiles[cid]['driver_type'] for cid in cluster_profiles.keys()]
        type_counts = {}
        for dtype in driver_types:
            type_counts[dtype] = type_counts.get(dtype, 0) + 1
        
        colors_pie = {'aggressive': 'red', 'dynamic': 'orange', 'cornering': 'purple', 
                      'efficient': 'green', 'cautious': 'blue', 'normal': 'cyan', 'unknown': 'gray'}
        pie_colors = [colors_pie.get(dtype, 'gray') for dtype in type_counts.keys()]
        
        plt.pie(type_counts.values(), labels=type_counts.keys(), colors=pie_colors,
                autopct='%1.1f%%', startangle=90)
        plt.title('Driver Type Distribution\n(Startup Phase)')
        
        # 6. 能耗效率对比
        ax6 = plt.subplot(3, 3, 6)
        cluster_ids = list(cluster_profiles.keys())
        energy_effs = [cluster_profiles[cid]['energy_efficiency'] for cid in cluster_ids]
        driver_types_list = [cluster_profiles[cid]['driver_type'] for cid in cluster_ids]
        
        bar_colors = [colors_pie.get(dtype, 'gray') for dtype in driver_types_list]
        bars = plt.bar(range(len(cluster_ids)), energy_effs, color=bar_colors, alpha=0.8)
        plt.title('Energy Efficiency by Cluster\n(Startup Phase)')
        plt.xlabel('Cluster')
        plt.ylabel('Energy Efficiency (kWh/100km)')
        plt.xticks(range(len(cluster_ids)), [f'C{cid}\n({dtype})' for cid, dtype in zip(cluster_ids, driver_types_list)])
        
        for bar, eff in zip(bars, energy_effs):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(energy_effs)*0.01, 
                    f'{eff:.1f}', ha='center', va='bottom')
        
        # 7. 速度vs加速度散点图
        ax7 = plt.subplot(3, 3, 7)
        scatter = plt.scatter(data['speed_kmh'], data['acceleration'], 
                             c=data['driver_cluster'], cmap='viridis', alpha=0.6, s=20)
        plt.title('Speed vs Acceleration\n(Startup Phase)')
        plt.xlabel('Speed (km/h)')
        plt.ylabel('Acceleration (m/s²)')
        plt.colorbar(scatter, label='Cluster')
        plt.grid(True, alpha=0.3)
        
        # 8. 时间序列加速度
        ax8 = plt.subplot(3, 3, 8)
        sample_size = min(200, len(data))
        sample_data = data.head(sample_size)
        sample_times = sample_data['relative_time_s'] / 60
        
        plt.plot(sample_times, sample_data['acceleration'], 'g-', alpha=0.7, linewidth=1)
        plt.title(f'Acceleration Time Series\n(First {sample_size} points)')
        plt.xlabel('Time (minutes)')
        plt.ylabel('Acceleration (m/s²)')
        plt.grid(True, alpha=0.3)
        
        # 9. 综合评分仪表盘
        ax9 = plt.subplot(3, 3, 9)
        
        # 计算评分 - 适应起步阶段
        total_events = len(hard_brake_events)
        total_time_hours = data['relative_time_s'].max() / 3600
        events_per_hour = total_events / max(total_time_hours, 0.1)
        
        avg_energy_eff = np.mean([p['energy_efficiency'] for p in cluster_profiles.values()])
        
        # 评分计算（针对起步阶段调整）
        safety_score = max(0, 100 - events_per_hour * 15)  # 起步阶段降低惩罚
        efficiency_score = max(0, 100 - max(0, avg_energy_eff - 25) * 2)  # 起步阶段能耗可能较高
        smoothness_score = 100 - min(50, np.mean([abs(p['avg_jerk']) for p in cluster_profiles.values()]) * 2)
        overall_score = (safety_score + efficiency_score + smoothness_score) / 3
        
        # 绘制仪表盘
        scores = [safety_score, efficiency_score, smoothness_score, overall_score]
        labels = ['Safety', 'Efficiency', 'Smoothness', 'Overall']
        colors_gauge = ['red', 'green', 'blue', 'purple']
        
        bars = plt.bar(labels, scores, color=colors_gauge, alpha=0.7)
        plt.ylim(0, 100)
        plt.title('Eco-Driving Score Dashboard\n(Startup Phase)')
        plt.ylabel('Score (0-100)')
        
        for bar, score in zip(bars, scores):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
                    f'{score:.0f}', ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        
        if save_plots:
            plt.savefig('eco_driving_startup_analysis.png', dpi=300, bbox_inches='tight')
            logger.info("Saved startup phase analysis plot")
        
        plt.show()
        
        return True
    
    def save_results(self, output_dir='eco_driving_results'):
        """保存分析结果"""
        if not self.results:
            logger.error("No results to save")
            return False
        
        # 创建输出目录
        os.makedirs(output_dir, exist_ok=True)
        
        try:
            # 保存主要数据
            self.results['data'].to_csv(f'{output_dir}/processed_data.csv', index=False)
            
            if not self.results['hard_brake_events'].empty:
                self.results['hard_brake_events'].to_csv(f'{output_dir}/hard_brake_events.csv', index=False)
            
            # 保存JSON报告
            with open(f'{output_dir}/analysis_report.json', 'w', encoding='utf-8') as f:
                json.dump(self.results['report'], f, indent=2, ensure_ascii=False, default=str)
            
            # 保存文本报告
            self._save_text_report(f'{output_dir}/eco_driving_report.txt')
            
            logger.info(f"Results saved to {output_dir}/")
            return True
            
        except Exception as e:
            logger.error(f"Error saving results: {e}")
            return False
    
    def _save_text_report(self, filepath):
        """保存详细文本报告"""
        with open(filepath, 'w', encoding='utf-8') as f:
            report = self.results['report']
            cluster_profiles = self.results['cluster_profiles']
            hard_brake_events = self.results['hard_brake_events']
            
            f.write("=" * 80 + "\n")
            f.write("ECO-DRIVING ANALYSIS REPORT - STARTUP PHASE\n")
            f.write("=" * 80 + "\n\n")
            
            # 基本信息
            f.write(f"Analysis Date: {report['metadata']['analysis_date']}\n")
            f.write(f"System Version: {report['metadata']['version']} (Microsecond Precision)\n")
            f.write(f"Data Points: {report['metadata']['data_points']}\n")
            f.write(f"Duration: {report['metadata']['duration_minutes']:.1f} minutes\n")
            f.write(f"Total Distance: {report['metadata']['total_distance_km']:.3f} km\n\n")
            
            # 驾驶摘要
            f.write("DRIVING SUMMARY (STARTUP PHASE):\n")
            f.write("-" * 40 + "\n")
            f.write(f"Maximum Speed: {report['driving_summary']['max_speed_kmh']:.1f} km/h\n")
            f.write(f"Average Speed: {report['driving_summary']['avg_speed_kmh']:.1f} km/h\n")
            f.write(f"Total Hard Brakes: {report['driving_summary']['total_hard_brakes']}\n")
            f.write(f"Hard Brakes per Hour: {report['driving_summary']['hard_brakes_per_hour']:.1f}\n\n")
            
            # 聚类分析
            f.write("DRIVER BEHAVIOR CLUSTERS (STARTUP PHASE):\n")
            f.write("-" * 40 + "\n")
            for cid, profile in cluster_profiles.items():
                f.write(f"\nCluster {cid} - {profile['driver_type'].upper()}:\n")
                f.write(f"  Sample Size: {profile['size']} points\n")
                f.write(f"  Average Speed: {profile['avg_speed']*3.6:.1f} km/h\n")
                f.write(f"  Average Acceleration: {profile['avg_acceleration']:.3f} m/s²\n")
                f.write(f"  Average Jerk: {profile['avg_jerk']:.3f} m/s³ (instantaneous)\n")
                f.write(f"  Average Turning Rate: {profile['avg_turning_rate']:.3f} rad/s\n")
                f.write(f"  Energy Efficiency: {profile['energy_efficiency']:.2f} kWh/100km\n")
                f.write(f"  Total Distance: {profile['total_distance_km']:.4f} km\n")
                f.write(f"  Total Time: {profile['total_time_s']:.1f} seconds\n")
            
            # 急刹车详情
            f.write(f"\nHARD BRAKE EVENTS DETAIL:\n")
            f.write("-" * 40 + "\n")
            if not hard_brake_events.empty:
                for i, (idx, event) in enumerate(hard_brake_events.iterrows()):
                    time_min = event['relative_time_s'] / 60
                    f.write(f"  Event {i+1}: {time_min:.1f} min - Speed: {event['speed_kmh']:.1f} km/h, "
                           f"Acceleration: {event['acceleration']:.2f} m/s²\n")
            else:
                f.write("  No hard brake events detected - Excellent smooth driving!\n")
            
            # 建议
            f.write(f"\nRECOMMENDATIONS:\n")
            f.write("-" * 40 + "\n")
            for i, rec in enumerate(report['recommendations']):
                f.write(f"{i+1}. [{rec['category']} - {rec['priority']}] {rec['message']}\n")
                f.write(f"   Impact: {rec['impact']}\n\n")
            
            # 技术说明
            f.write("TECHNICAL NOTES:\n")
            f.write("-" * 20 + "\n")
            f.write("• Timestamp precision: Microseconds\n")
            f.write("• Analysis focus: Vehicle startup phase\n")
            f.write("• Jerk calculation: Instantaneous values preserved\n")
            f.write("• Acceleration: Real GPS-based calculation with synthetic backup\n")
            f.write("• Energy model: Adapted for low-speed urban driving\n")

# 主要使用接口
def run_complete_eco_driving_analysis(data_file, config=None):
    """
    运行完整的生态驾驶分析 - 微秒时间戳版本
    
    Args:
        data_file: 传感器数据文件路径
        config: 可选配置字典
    
    Returns:
        EcoDrivingAnalyzer实例，包含所有分析结果
    """
    print("🚗 Starting Complete Eco-Driving Analysis System v2.1")
    print("📍 Optimized for startup phase analysis with microsecond precision")
    print("=" * 70)
    
    # 初始化分析器
    analyzer = EcoDrivingAnalyzer(config)
    
    # 1. 解析数据
    print("📊 Step 1: Parsing sensor data (microsecond timestamps)...")
    if not analyzer.parse_sensor_data(data_file):
        print("❌ Failed to parse sensor data")
        return None
    
    # 2. 提取特征
    print("🔍 Step 2: Extracting driving features...")
    if not analyzer.extract_driving_features():
        print("❌ Failed to extract features")
        return None
    
    # 3. 数据同步和能耗建模
    print("⚡ Step 3: Synchronizing data and modeling energy...")
    combined_data = analyzer.synchronize_and_model_energy()
    if combined_data.empty:
        print("❌ Failed to synchronize data")
        return None
    
    # 4. 生成综合报告
    print("📋 Step 4: Generating comprehensive report...")
    report = analyzer.generate_comprehensive_report()
    if not report:
        print("❌ Failed to generate report")
        return None
    
    # 5. 可视化结果
    print("📈 Step 5: Creating visualizations...")
    analyzer.visualize_results()
    
    # 6. 保存结果
    print("💾 Step 6: Saving results...")
    analyzer.save_results()
    
    print("\n✅ Analysis Complete!")
    print("=" * 70)
    
    # 打印关键结果
    print(f"📊 Key Results (Startup Phase):")
    print(f"  • Duration: {report['metadata']['duration_minutes']:.1f} minutes")
    print(f"  • Max Speed: {report['driving_summary']['max_speed_kmh']:.1f} km/h")
    print(f"  • Hard Brake Events: {len(analyzer.results['hard_brake_events'])}")
    print(f"  • Driver Behavior Types: {len(set(p['driver_type'] for p in analyzer.results['cluster_profiles'].values()))}")
    print(f"  • Average Energy Efficiency: {np.mean([p['energy_efficiency'] for p in analyzer.results['cluster_profiles'].values()]):.1f} kWh/100km")
    
    return analyzer

# 使用示例和测试代码
if __name__ == "__main__":
    # 针对起步阶段优化的配置
    startup_config = {
        'hard_brake_threshold': -2.0,  # 降低阈值适应起步
        'speed_drop_threshold': 6,     # 降低速度下降阈值
        'min_brake_speed': 8,          # 降低最小刹车速度
        'energy_model': {
            'vehicle_mass': 1600,      # 紧凑型轿车
            'drag_coefficient': 0.26,  # 现代轿车优秀风阻
            'frontal_area': 2.2,       # 紧凑型车迎风面积
            'rolling_resistance': 0.010 # 优质轮胎
        },
        'clustering': {
            'n_clusters': 3,
            'features': ['speed', 'acceleration', 'jerk', 'turning_rate']
        }
    }
    
    # 运行分析
    data_file = "ts_1747221572.csv"  # 你的数据文件
    
    print("🚀 Running Eco-Driving Analysis - Startup Phase Optimized")
    print(f"📁 Data file: {data_file}")
    print("🔧 Configuration: Optimized for startup phase with microsecond precision")
    print("⏱️  Expected analysis time: 30-60 seconds")
    print()
    
    # 执行完整分析
    analyzer = run_complete_eco_driving_analysis(data_file, startup_config)
    
    if analyzer:
        print("\n🎉 Analysis completed successfully!")
        print("📁 Check 'eco_driving_results' folder for detailed outputs:")
        print("   • processed_data.csv - Complete processed dataset")
        print("   • hard_brake_events.csv - Hard brake event details") 
        print("   • analysis_report.json - Structured analysis results")
        print("   • eco_driving_report.txt - Human-readable report")
        print("   • eco_driving_startup_analysis.png - Comprehensive visualization")
        
        # 显示快速摘要
        print(f"\n🔍 Quick Analysis Summary:")
        report = analyzer.results['report']
        print(f"  ⏱️  Duration: {report['metadata']['duration_minutes']:.1f} minutes")
        print(f"  🚗 Max Speed: {report['driving_summary']['max_speed_kmh']:.1f} km/h")
        print(f"  🚨 Hard Brakes: {report['driving_summary']['total_hard_brakes']}")
        
        # 显示驾驶类型分布
        cluster_types = [p['driver_type'] for p in analyzer.results['cluster_profiles'].values()]
        type_counts = {}
        for dtype in cluster_types:
            type_counts[dtype] = type_counts.get(dtype, 0) + 1
        
        print(f"  🎯 Driving Types Found: {', '.join(type_counts.keys())}")
        
        # 显示主要建议
        print(f"\n💡 Top Recommendations:")
        for i, rec in enumerate(report['recommendations'][:2]):
            print(f"  {i+1}. [{rec['priority']}] {rec['message'][:80]}...")
        
        # 数据质量报告
        data = analyzer.results['data']
        print(f"\n📊 Data Quality Summary:")
        print(f"  • Total Data Points: {len(data):,}")
        print(f"  • GPS-Accelerometer Sync: {(data['sync_error_us'] < 100000).sum()}/{len(data)} points < 100ms")
        print(f"  • Acceleration Range: {data['acceleration'].min():.2f} to {data['acceleration'].max():.2f} m/s²")
        print(f"  • Jerk Range: {data['jerk'].min():.2f} to {data['jerk'].max():.2f} m/s³")
        print(f"  • Speed Range: {data['speed_kmh'].min():.1f} to {data['speed_kmh'].max():.1f} km/h")
        
        # 性能评估
        if 'hard_brake_events' in analyzer.results:
            safety_rating = "🟢 Excellent" if len(analyzer.results['hard_brake_events']) == 0 else \
                           "🟡 Good" if len(analyzer.results['hard_brake_events']) <= 2 else \
                           "🟠 Needs Improvement"
            print(f"  • Safety Rating: {safety_rating}")
        
        avg_energy = np.mean([p['energy_efficiency'] for p in analyzer.results['cluster_profiles'].values()])
        efficiency_rating = "🟢 Excellent" if avg_energy < 20 else \
                           "🟡 Good" if avg_energy < 30 else \
                           "🟠 Moderate"
        print(f"  • Efficiency Rating: {efficiency_rating} ({avg_energy:.1f} kWh/100km)")
        
        print(f"\n✨ Analysis Features Highlights:")
        print(f"  ✅ Microsecond timestamp precision")
        print(f"  ✅ Real GPS-based acceleration calculation")
        print(f"  ✅ Instantaneous jerk values (not averaged)")
        print(f"  ✅ Startup phase optimized thresholds")
        print(f"  ✅ Intelligent hard brake detection")
        print(f"  ✅ Multi-dimensional driver behavior clustering")
        print(f"  ✅ Personalized driving recommendations")
        
    else:
        print("❌ Analysis failed. Please check your data file and try again.")
        print("\n🔧 Troubleshooting Tips:")
        print("  • Ensure your CSV file exists and is readable")
        print("  • Check that the data format matches the expected structure:")
        print("    - GPS: 0,timestamp,lat,lon,alt,speed_kmh,satellites")
        print("    - Accelerometer: 1,timestamp,timestamp_us,x,y,z")
        print("    - Gyroscope: 2,timestamp,timestamp_us,x,y,z")
        print("  • Verify that timestamps are in microseconds")
        print("  • Make sure the file contains sufficient data points")

# 辅助函数：数据格式验证
def validate_data_format(file_path, sample_lines=10):
    """
    验证数据文件格式
    Args:
        file_path: 数据文件路径
        sample_lines: 检查的样本行数
    """
    print(f"🔍 Validating data format for {file_path}")
    print("=" * 50)
    
    try:
        with open(file_path, 'r') as f:
            lines = [f.readline().strip() for _ in range(sample_lines)]
        
        format_counts = {'gps': 0, 'acc': 0, 'gyro': 0, 'rot': 0, 'unknown': 0}
        
        print("Sample data lines:")
        for i, line in enumerate(lines):
            if not line:
                continue
            values = line.split(',')
            print(f"  {i+1}: {line}")
            
            if len(values) >= 2:
                try:
                    data_type = int(values[0])
                    if data_type == 0:
                        format_counts['gps'] += 1
                        if len(values) < 6:
                            print(f"    ⚠️  GPS line has only {len(values)} columns, expected 6+")
                    elif data_type == 1:
                        format_counts['acc'] += 1
                        if len(values) < 5:
                            print(f"    ⚠️  Accelerometer line has only {len(values)} columns, expected 5+")
                    elif data_type == 2:
                        format_counts['gyro'] += 1
                        if len(values) < 5:
                            print(f"    ⚠️  Gyroscope line has only {len(values)} columns, expected 5+")
                    elif data_type == 3:
                        format_counts['rot'] += 1
                        if len(values) < 6:
                            print(f"    ⚠️  Rotation line has only {len(values)} columns, expected 6+")
                    else:
                        format_counts['unknown'] += 1
                        print(f"    ⚠️  Unknown data type: {data_type}")
                except ValueError:
                    format_counts['unknown'] += 1
                    print(f"    ❌ Invalid data type: {values[0]}")
        
        print(f"\nData type distribution in sample:")
        for dtype, count in format_counts.items():
            if count > 0:
                print(f"  {dtype}: {count} lines")
        
        # 验证时间戳
        if format_counts['gps'] > 0 or format_counts['acc'] > 0:
            print(f"\nTimestamp validation:")
            for line in lines:
                if not line:
                    continue
                values = line.split(',')
                if len(values) >= 2:
                    try:
                        timestamp = int(values[1])
                        if timestamp > 1000000000000:  # 微秒级时间戳
                            print(f"  ✅ Microsecond timestamp detected: {timestamp}")
                        elif timestamp > 1000000000:   # 毫秒级时间戳
                            print(f"  ⚠️  Millisecond timestamp detected: {timestamp}")
                        else:
                            print(f"  ❌ Unusual timestamp: {timestamp}")
                        break
                    except ValueError:
                        continue
        
        return True
        
    except Exception as e:
        print(f"❌ Error validating file: {e}")
        return False

2025-05-27 15:22:06,499 - INFO - EcoDrivingAnalyzer v2.1 initialized (microsecond timestamps)
2025-05-27 15:22:06,499 - INFO - Parsing sensor data from ts_1747221572.csv


🚀 Running Eco-Driving Analysis - Startup Phase Optimized
📁 Data file: ts_1747221572.csv
🔧 Configuration: Optimized for startup phase with microsecond precision
⏱️  Expected analysis time: 30-60 seconds

🚗 Starting Complete Eco-Driving Analysis System v2.1
📍 Optimized for startup phase analysis with microsecond precision
📊 Step 1: Parsing sensor data (microsecond timestamps)...


2025-05-27 15:22:06,978 - INFO - Parsed 100000 lines from ts_1747221572.csv
2025-05-27 15:22:07,134 - INFO - Extracting driving features with microsecond precision



=== GPS数据解析 ===
GPS原始行数: 667
样本数据: ['0', '1747221671', '57.687946', '11.980719', '56.200001', '2.857636', '12']
GPS数据质量检查:
  数据点数: 667
  时间跨度: 0.0 秒 (0.0 分钟)
  速度范围: 0.0 - 91.8 km/h
  位置变化: 纬度 0.038410°, 经度 0.017737°

=== 加速度传感器数据解析 ===
加速度原始行数: 84489
样本数据: ['1', '1747221671', '3809', '-1.800781', '-0.957031', '9.539062']
加速度数据质量检查:
  数据点数: 84489
  时间跨度: 1.0 秒
  原始加速度最大值: 14.90

=== 陀螺仪数据解析 ===
  数据点数: 12051
  Z轴角速度范围: -0.529 - 0.596 rad/s
🔍 Step 2: Extracting driving features...

=== GPS特征提取 ===
GPS加速度计算结果:
  非零值: 0/667 (0.0%)
  范围: 0.000 - 0.000 m/s²
  平均: 0.000 m/s²
  标准差: 0.000 m/s²
⚠️  GPS加速度计算异常，使用改进的合成方法...
  合成加速度范围: -12.000 - 12.000 m/s²

=== 加速度传感器特征提取 ===
原始加速度数据统计:
  X: -5.055 - 2.758
  Y: -5.785 - 4.367
  Z: 3.523 - 14.902
Magnitude统计: 4.599 - 14.913
Magnitude变化统计: -5.348178 - 5.949456
时间间隔统计: 0.000000 - 0.000221 秒


2025-05-27 15:22:07,509 - INFO - Synchronizing data and modeling energy consumption (microsecond precision)


Jerk计算结果:
  原始范围: -500.000 - 500.000 m/s³
  平滑后范围: 0.000 - 0.000 m/s³
  非零值: 96/84489
  99%分位数: ±0.000 m/s³

=== 陀螺仪特征提取 ===
陀螺仪原始数据:
  X: -0.264 - 0.252
  Y: -0.387 - 0.428
  Z: -0.529 - 0.596
转向率统计: 0.000 - 0.596
⚡ Step 3: Synchronizing data and modeling energy...

=== 数据同步 (微秒精度) ===
GPS数据点: 667
加速度数据点: 84489


AttributeError: 'EcoDrivingAnalyzer' object has no attribute '_calculate_energy_consumption'

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from scipy.signal import savgol_filter
from geopy.distance import geodesic
import warnings
import logging
from datetime import datetime
import json
import os

# 配置日志和常量
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
warnings.filterwarnings('ignore')

# 系统常量 - 修正为微秒
GRAVITY = 9.81
MAX_TIME_DIFF = 5000000  # 最大时间差（微秒）5秒
MAX_SAMPLES = 100000     # 增加最大样本数
VERSION = "2.1"

class EcoDrivingAnalyzer:
    """生态驾驶分析主类 - 微秒时间戳版本"""
    
    def __init__(self, config=None):
        """初始化分析器"""
        self.config = config or self._default_config()
        self.raw_data = {}
        self.processed_data = None
        self.features = {}
        self.results = {}
        self.time_offset = None  # 用于计算相对时间
        
        logger.info(f"EcoDrivingAnalyzer v{VERSION} initialized (microsecond timestamps)")
    
    def _default_config(self):
        """默认配置"""
        return {
            'hard_brake_threshold': -2.5,
            'speed_drop_threshold': 8,
            'min_brake_speed': 15,
            'energy_model': {
                'vehicle_mass': 1800,          # 更轻的乘用车
                'drag_coefficient': 0.28,      # 现代轿车风阻系数
                'frontal_area': 2.3,           # 现代轿车迎风面积
                'rolling_resistance': 0.012    # 现代轮胎滚动阻力
            },
            'clustering': {
                'n_clusters': 3,
                'features': ['speed', 'acceleration', 'jerk', 'turning_rate']
            }
        }
    
    def parse_sensor_data(self, file_path):
        """解析传感器数据文件 - 修正微秒时间戳处理"""
        logger.info(f"Parsing sensor data from {file_path}")
        
        gps_rows, acc_rows, gyro_rows, rot_rows = [], [], [], []
        total_rows = 0
        
        try:
            with open(file_path, "r") as file:
                for line_num, line in enumerate(file):
                    values = line.strip().split(",")
                    if len(values) < 2:
                        continue
                    
                    try:
                        data_type = int(values[0])
                    except ValueError:
                        continue
                    
                    # 修正数据解析逻辑
                    if data_type == 0 and len(values) >= 6:  # GPS数据
                        gps_rows.append(values)
                    elif data_type == 1 and len(values) >= 5:  # 加速度计数据
                        acc_rows.append(values)
                    elif data_type == 2 and len(values) >= 5:  # 陀螺仪数据
                        gyro_rows.append(values)
                    elif data_type == 3 and len(values) >= 6:  # 四元数数据
                        rot_rows.append(values)
                    
                    total_rows += 1
                    if total_rows >= MAX_SAMPLES:
                        break
            
            logger.info(f"Parsed {total_rows} lines from {file_path}")
        
        except Exception as e:
            logger.error(f"Error reading file: {e}")
            return False
        
        # 处理GPS数据
        if gps_rows:
            print(f"\n=== GPS数据解析 ===")
            print(f"GPS原始行数: {len(gps_rows)}")
            print(f"样本数据: {gps_rows[0]}")
            
            # 处理GPS数据格式：0,timestamp,lat,lon,alt,speed_kmh,satellites
            gps_data = []
            for row in gps_rows:
                try:
                    if len(row) >= 6:
                        gps_data.append([
                            int(row[0]),      # data_type
                            int(row[1]),      # timestamp (微秒)
                            float(row[2]),    # latitude
                            float(row[3]),    # longitude
                            float(row[4]),    # altitude
                            float(row[5]),    # speed_kmh
                            int(row[6]) if len(row) > 6 else 12  # satellites
                        ])
                except (ValueError, IndexError) as e:
                    continue
            
            if gps_data:
                gps_df = pd.DataFrame(gps_data, columns=[
                    "data_type", "timestamp", "latitude", "longitude", 
                    "altitude", "speed_kmh", "satellites"
                ])
                
                gps_df["speed_ms"] = gps_df["speed_kmh"] / 3.6
                
                # 设置时间偏移量
                if self.time_offset is None:
                    self.time_offset = gps_df['timestamp'].min()
                
                # 计算相对时间（秒）
                gps_df['relative_time_s'] = (gps_df['timestamp'] - self.time_offset) / 1000000.0
                
                print(f"GPS数据质量检查:")
                print(f"  数据点数: {len(gps_df)}")
                print(f"  时间跨度: {gps_df['relative_time_s'].max():.1f} 秒 ({gps_df['relative_time_s'].max()/60:.1f} 分钟)")
                print(f"  速度范围: {gps_df['speed_kmh'].min():.1f} - {gps_df['speed_kmh'].max():.1f} km/h")
                print(f"  位置变化: 纬度 {gps_df['latitude'].max()-gps_df['latitude'].min():.6f}°, 经度 {gps_df['longitude'].max()-gps_df['longitude'].min():.6f}°")
                
                # 数据清洗
                gps_df = gps_df[gps_df["speed_kmh"] <= 200]
                gps_df = gps_df[gps_df["latitude"].between(-90, 90)]
                gps_df = gps_df[gps_df["longitude"].between(-180, 180)]
                
                self.raw_data['gps'] = gps_df
        
        # 处理加速度传感器数据
        if acc_rows:
            print(f"\n=== 加速度传感器数据解析 ===")
            print(f"加速度原始行数: {len(acc_rows)}")
            print(f"样本数据: {acc_rows[0]}")
            
            # 处理加速度数据格式：1,timestamp,timestamp_us,x,y,z
            acc_data = []
            for row in acc_rows:
                try:
                    if len(row) >= 5:
                        acc_data.append([
                            int(row[0]),      # data_type
                            int(row[1]),      # timestamp (微秒)
                            int(row[2]),      # timestamp_us (额外的微秒)
                            float(row[3]),    # x
                            float(row[4]),    # y
                            float(row[5]) if len(row) > 5 else 0.0  # z
                        ])
                except (ValueError, IndexError):
                    continue
            
            if acc_data:
                acc_df = pd.DataFrame(acc_data, columns=[
                    "data_type", "timestamp", "timestamp_us", "x", "y", "z"
                ])
                
                # 计算完整时间戳（主时间戳 + 微秒偏移）
                acc_df['full_timestamp'] = acc_df['timestamp'] + acc_df['timestamp_us']
                
                if self.time_offset is None:
                    self.time_offset = acc_df['full_timestamp'].min()
                
                acc_df['relative_time_s'] = (acc_df['full_timestamp'] - self.time_offset) / 1000000.0
                
                print(f"加速度数据质量检查:")
                print(f"  数据点数: {len(acc_df)}")
                print(f"  时间跨度: {acc_df['relative_time_s'].max():.1f} 秒")
                
                # 检查加速度范围和缩放
                max_acc = max(acc_df['x'].abs().max(), acc_df['y'].abs().max(), acc_df['z'].abs().max())
                print(f"  原始加速度最大值: {max_acc:.2f}")
                
                # 智能缩放 - 根据数据范围判断单位
                if max_acc > 50:  # 如果数值很大，可能需要缩放
                    if max_acc > 1000:
                        scale_factor = 1000.0
                        print("  检测到加速度单位可能是 mg，应用1000x缩放")
                    else:
                        scale_factor = 10.0
                        print("  检测到加速度单位异常，应用10x缩放")
                    
                    acc_df[["x", "y", "z"]] = acc_df[["x", "y", "z"]] / scale_factor
                    print(f"  缩放后范围: {acc_df['x'].min():.2f} - {acc_df['x'].max():.2f}")
                
                acc_df["magnitude"] = np.sqrt(acc_df["x"]**2 + acc_df["y"]**2 + acc_df["z"]**2)
                self.raw_data['acc'] = acc_df
        
        # 处理陀螺仪数据
        if gyro_rows:
            print(f"\n=== 陀螺仪数据解析 ===")
            gyro_data = []
            for row in gyro_rows:
                try:
                    if len(row) >= 5:
                        gyro_data.append([
                            int(row[0]),      # data_type
                            int(row[1]),      # timestamp
                            int(row[2]),      # timestamp_us
                            float(row[3]),    # x
                            float(row[4]),    # y
                            float(row[5]) if len(row) > 5 else 0.0  # z
                        ])
                except (ValueError, IndexError):
                    continue
            
            if gyro_data:
                gyro_df = pd.DataFrame(gyro_data, columns=[
                    "data_type", "timestamp", "timestamp_us", "x", "y", "z"
                ])
                gyro_df['full_timestamp'] = gyro_df['timestamp'] + gyro_df['timestamp_us']
                
                if self.time_offset is None:
                    self.time_offset = gyro_df['full_timestamp'].min()
                
                gyro_df['relative_time_s'] = (gyro_df['full_timestamp'] - self.time_offset) / 1000000.0
                gyro_df["turning_rate"] = gyro_df["z"].abs()
                
                print(f"  数据点数: {len(gyro_df)}")
                print(f"  Z轴角速度范围: {gyro_df['z'].min():.3f} - {gyro_df['z'].max():.3f} rad/s")
                
                self.raw_data['gyro'] = gyro_df
        
        return True
    
    def extract_driving_features(self):
        """提取驾驶特征 - 修正时间处理"""
        logger.info("Extracting driving features with microsecond precision")
        
        # GPS路线特征
        if 'gps' in self.raw_data and not self.raw_data['gps'].empty:
            gps_df = self.raw_data['gps'].copy().sort_values('timestamp').reset_index(drop=True)
            
            print(f"\n=== GPS特征提取 ===")
            
            # 计算距离和坡度
            distances, gradients = [], []
            for i in range(1, len(gps_df)):
                p1 = (gps_df["latitude"].iloc[i-1], gps_df["longitude"].iloc[i-1])
                p2 = (gps_df["latitude"].iloc[i], gps_df["longitude"].iloc[i])
                try:
                    d = geodesic(p1, p2).meters
                except Exception:
                    d = 0
                distances.append(d)
                
                alt_diff = gps_df["altitude"].iloc[i] - gps_df["altitude"].iloc[i-1]
                gradient = alt_diff / d if d > 0.5 else 0  # 距离>0.5米才计算坡度
                gradients.append(gradient)
            
            if distances:
                gps_df.loc[1:, "distance"] = distances
                gps_df.loc[1:, "gradient"] = gradients
            gps_df["distance"] = gps_df.get("distance", 0).fillna(0)
            gps_df["gradient"] = gps_df.get("gradient", 0).fillna(0)
            
            # 修正的加速度计算 - 使用微秒精度
            dt_values = gps_df["timestamp"].diff() / 1000000.0  # 转换为秒
            speed_changes_ms = gps_df["speed_ms"].diff()
            
            accelerations = []
            for i in range(len(gps_df)):
                if i == 0:
                    accelerations.append(0.0)
                else:
                    dt = dt_values.iloc[i]
                    speed_change = speed_changes_ms.iloc[i]
                    
                    # 修正时间范围检查
                    if 0.01 < dt < 300:  # 10ms 到 5分钟
                        acc = speed_change / dt
                        acc = max(-12, min(12, acc))  # 合理的加速度范围
                        accelerations.append(acc)
                    else:
                        # 对于异常时间间隔，使用前一个加速度的衰减值
                        if i > 1:
                            accelerations.append(accelerations[-1] * 0.8)
                        else:
                            accelerations.append(0.0)
            
            gps_df["acceleration"] = accelerations
            
            # 检查GPS加速度计算结果
            acc_nonzero = (np.array(accelerations) != 0).sum()
            acc_range = [min(accelerations), max(accelerations)]
            
            print(f"GPS加速度计算结果:")
            print(f"  非零值: {acc_nonzero}/{len(accelerations)} ({acc_nonzero/len(accelerations)*100:.1f}%)")
            print(f"  范围: {acc_range[0]:.3f} - {acc_range[1]:.3f} m/s²")
            print(f"  平均: {np.mean(accelerations):.3f} m/s²")
            print(f"  标准差: {np.std(accelerations):.3f} m/s²")
            
            # 如果GPS加速度仍然问题很大，使用改进的合成方法
            if abs(max(accelerations)) < 0.01 or acc_nonzero < len(accelerations) * 0.1:
                print("⚠️  GPS加速度计算异常，使用改进的合成方法...")
                
                # 使用速度的变化趋势生成更真实的加速度
                speed_smooth = gps_df['speed_ms'].rolling(window=3, center=True).mean().fillna(gps_df['speed_ms'])
                speed_diff = speed_smooth.diff()
                time_diff = gps_df['relative_time_s'].diff()
                
                synthetic_acc = []
                for i in range(len(gps_df)):
                    if i == 0:
                        synthetic_acc.append(0.0)
                    else:
                        dt = time_diff.iloc[i]
                        if dt > 0:
                            acc = speed_diff.iloc[i] / dt
                            acc = max(-8, min(8, acc))
                            synthetic_acc.append(acc)
                        else:
                            synthetic_acc.append(0.0)
                
                gps_df['acceleration'] = synthetic_acc
                print(f"  合成加速度范围: {min(synthetic_acc):.3f} - {max(synthetic_acc):.3f} m/s²")
            
            self.features['route'] = gps_df
        
        # 加速度传感器特征 - 修正时间处理
        if 'acc' in self.raw_data and not self.raw_data['acc'].empty:
            acc_df = self.raw_data['acc'].copy().sort_values('full_timestamp').reset_index(drop=True)
            
            print(f"\n=== 加速度传感器特征提取 ===")
            
            # 计算Jerk（急动度）- 使用微秒精度
            dt_acc = acc_df["full_timestamp"].diff() / 1000000.0  # 转换为秒
            magnitude_changes = acc_df["magnitude"].diff()

            # 调试：检查magnitude变化
            print(f"Magnitude统计: {acc_df['magnitude'].min():.3f} - {acc_df['magnitude'].max():.3f}")
            print(f"Magnitude变化统计: {magnitude_changes.min():.6f} - {magnitude_changes.max():.6f}")
            print(f"时间间隔统计: {dt_acc.min():.6f} - {dt_acc.max():.6f} 秒")
            
            for i in range(len(acc_df)):
                if i == 0:
                    jerks.append(0.0)
                else:
                    dt = dt_acc.iloc[i]
                    mag_change = magnitude_changes.iloc[i]
                    
                    # 修正时间范围 - 加速度计采样频率更高
                    if 0.001 < dt < 1.0:  # 1ms 到 1秒
                        jerk = mag_change / dt
                        jerk = max(-100, min(100, jerk))  # 扩大jerk范围
                        jerks.append(jerk)
                    else:
                        jerks.append(0.0)
            
            acc_df["jerk"] = jerks
            
            # 改进的平滑处理
            if len(acc_df) > 5:
                # 使用更小的窗口保持瞬时特性
                win = min(3, len(acc_df)//2*2+1)
                if win >= 3:
                    acc_df["jerk_smooth"] = savgol_filter(acc_df["jerk"], win, 1)
                else:
                    acc_df["jerk_smooth"] = acc_df["jerk"]
            else:
                acc_df["jerk_smooth"] = acc_df["jerk"]
            
            print(f"Jerk计算结果:")
            print(f"  原始范围: {min(jerks):.3f} - {max(jerks):.3f} m/s³")
            print(f"  平滑后范围: {acc_df['jerk_smooth'].min():.3f} - {acc_df['jerk_smooth'].max():.3f} m/s³")
            print(f"  非零值: {(np.array(jerks) != 0).sum()}/{len(jerks)}")
            
            self.features['behavior'] = acc_df
        
        # 陀螺仪转向特征
        if 'gyro' in self.raw_data and not self.raw_data['gyro'].empty:
            self.features['turning'] = self.raw_data['gyro'].copy()
        
        return True
    
    def detect_hard_brake_events(self):
        """检测急刹车事件 - 修正版"""
        logger.info("Detecting hard brake events with improved algorithm")
        
        if 'route' not in self.features:
            logger.warning("No route data available for hard brake detection")
            return pd.DataFrame()
        
        df = self.features['route'].copy()
        
        # 数据平滑
        if len(df) > 5:
            window_size = min(5, len(df) // 2 * 2 + 1)
            df['speed_smooth'] = savgol_filter(df['speed_kmh'], window_size, 2)
        else:
            df['speed_smooth'] = df['speed_kmh']
        
        # 速度变化分析 - 使用相对时间
        df['speed_change'] = df['speed_smooth'].diff()
        time_diff_s = df['relative_time_s'].diff()
        df['speed_change_rate'] = df['speed_change'] / time_diff_s
        df['speed_change_pct'] = df['speed_change'] / (df['speed_smooth'].shift(1) + 1)
        
        # 针对低速场景调整的急刹车检测条件
        conditions = [
            # 条件1: 适应低速的绝对速度下降
            (df['speed_change'] < -5) & (df['speed_smooth'] > 10),  # 降低速度阈值
            
            # 条件2: 相对速度变化
            (df['speed_change_pct'] < -0.20) & (df['speed_smooth'] > 8),  # 提高相对变化敏感度
            
            # 条件3: 速度变化率（针对起步阶段）
            (df['speed_change_rate'] < -8) & (df['speed_smooth'] > 5),  # 降低速度阈值
        ]
        
        df['hard_brake_candidate'] = np.logical_or.reduce(conditions)
        
        # 速度峰值检测 - 针对低速优化
        df['is_speed_peak'] = False
        df['hard_brake_refined'] = False
        
        # 寻找局部速度峰值（降低阈值适应低速）
        for i in range(2, len(df) - 2):
            current_speed = df['speed_smooth'].iloc[i]
            prev_speed = df['speed_smooth'].iloc[i-1]
            next_speed = df['speed_smooth'].iloc[i+1]
            
            if (current_speed > prev_speed and current_speed > next_speed and current_speed > 8):  # 降低峰值阈值
                df.loc[i, 'is_speed_peak'] = True
        
        # 在峰值后检测急刹车
        for i in range(len(df)):
            if df['is_speed_peak'].iloc[i]:
                peak_speed = df['speed_smooth'].iloc[i]
                
                for j in range(i+1, min(i+6, len(df))):
                    current_speed = df['speed_smooth'].iloc[j]
                    speed_drop = peak_speed - current_speed
                    time_span = df['relative_time_s'].iloc[j] - df['relative_time_s'].iloc[i]
                    
                    # 针对低速调整的急刹车条件
                    if speed_drop > 8 and time_span < 5:  # 降低速度下降阈值
                        df.loc[j, 'hard_brake_refined'] = True
                        break
        
        # 最终急刹车事件
        df['hard_brake_final'] = df['hard_brake_candidate'] | df['hard_brake_refined']
        
        # 去重处理
        df['hard_brake_filtered'] = False
        if df['hard_brake_final'].any():
            brake_indices = df[df['hard_brake_final']].index
            filtered_indices = [brake_indices[0]] if len(brake_indices) > 0 else []
            
            for i in range(1, len(brake_indices)):
                time_gap = df['relative_time_s'].iloc[brake_indices[i]] - df['relative_time_s'].iloc[brake_indices[i-1]]
                if time_gap > 2.0:  # 2秒间隔
                    filtered_indices.append(brake_indices[i])
            
            if filtered_indices:
                df.loc[filtered_indices, 'hard_brake_filtered'] = True
        
        hard_brake_events = df[df['hard_brake_filtered']]
        
        print(f"\n=== 急刹车检测结果 ===")
        print(f"候选事件: {df['hard_brake_candidate'].sum()}")
        print(f"精炼事件: {df['hard_brake_refined'].sum()}")
        print(f"最终检测到: {len(hard_brake_events)} 个急刹车事件")
        
        if not hard_brake_events.empty:
            print("急刹车事件详情:")
            for i, (idx, event) in enumerate(hard_brake_events.iterrows()):
                time_min = event['relative_time_s'] / 60
                print(f"  事件{i+1}: {time_min:.1f}分钟 - 速度: {event['speed_kmh']:.1f} km/h, 加速度: {event['acceleration']:.2f} m/s²")
        
        # 更新路线数据
        self.features['route'] = df
        
        return hard_brake_events
    
    def synchronize_and_model_energy(self):
        """数据同步和能耗建模 - 微秒精度版本"""
        logger.info("Synchronizing data and modeling energy consumption (microsecond precision)")
        
        if 'route' not in self.features or 'behavior' not in self.features:
            logger.warning("Missing required data for energy modeling")
            return pd.DataFrame()
        
        route_df = self.features['route']
        behavior_df = self.features['behavior']
        
        # 使用微秒时间戳进行同步
        route_times = route_df['timestamp'].values
        behavior_times = behavior_df['full_timestamp'].values
        
        print(f"\n=== 数据同步 (微秒精度) ===")
        print(f"GPS数据点: {len(route_df)}")
        print(f"加速度数据点: {len(behavior_df)}")
        
        combined_data = []
        sync_errors = 0
        perfect_matches = 0
        
        for i, row in route_df.iterrows():
            gps_time = row['timestamp']
            time_diffs = np.abs(behavior_times - gps_time)
            closest_acc_idx = np.argmin(time_diffs)
            min_time_diff = time_diffs[closest_acc_idx]
            
            if min_time_diff == 0:
                perfect_matches += 1
            elif min_time_diff > MAX_TIME_DIFF:
                sync_errors += 1
                continue
            
            entry = {
                'timestamp': gps_time,
                'relative_time_s': row['relative_time_s'],
                'speed': row['speed_ms'],
                'speed_kmh': row['speed_kmh'],
                'acceleration': row['acceleration'],
                'gradient': row['gradient'],
                'jerk': behavior_df['jerk_smooth'].iloc[closest_acc_idx],
                'sync_error_us': min_time_diff  # 微秒
            }
            
            # 添加转向数据
            if 'turning' in self.features:
                turning_times = self.features['turning']['full_timestamp'].values
                closest_turn_idx = np.argmin(np.abs(turning_times - gps_time))
                turn_time_diff = np.abs(turning_times[closest_turn_idx] - gps_time)
                
                if turn_time_diff <= MAX_TIME_DIFF:
                    entry['turning_rate'] = self.features['turning']['turning_rate'].iloc[closest_turn_idx]
                else:
                    entry['turning_rate'] = 0
            else:
                entry['turning_rate'] = 0
            
            combined_data.append(entry)
        
        if not combined_data:
            logger.warning("No synchronized data points found")
            return pd.DataFrame()
        
        combined_df = pd.DataFrame(combined_data)
        
        # 能耗计算
        combined_df['energy_factor'] = self._calculate_energy_consumption(combined_df)
        
        print(f"同步结果:")
        print(f"  完美匹配: {perfect_matches}")
        print(f"  成功同步: {len(combined_data)}")
        print(f"  同步错误: {sync_errors}")
        print(f"  平均同步误差: {combined_df['sync_error_us'].mean():.0f} 微秒")
        
        self.processed_data = combined_df
        return combined_df
    
    def _calculate_energy_consumption(self, df):
        """计算能耗 - 修正版"""
        config = self.config['energy_model']
        mass = config['vehicle_mass']
        g = GRAVITY
        rho = 1.225  # 空气密度
        Cd = config['drag_coefficient']
        A = config['frontal_area']
        Cr = config['rolling_resistance']
        
        power_W = np.zeros(len(df))
        
        for i in range(len(df)):
            v = max(0.1, df['speed'].iloc[i])
            θ = np.arctan(df['gradient'].iloc[i]) if abs(df['gradient'].iloc[i]) < 1 else 0
            
            F_roll = Cr * mass * g * np.cos(θ)
            F_aero = 0.5 * rho * Cd * A * v**2
            F_grad = mass * g * np.sin(θ)
            F_acc = mass * df['acceleration'].iloc[i]
            
            F_total = F_roll + F_aero + F_grad + F_acc
            F_traction = max(F_total, 0)
            power_W[i] = F_traction * v
        
        # 计算能耗 - 使用相对时间
        if len(df) > 1:
            dt = df['relative_time_s'].diff().fillna(1.0)
            dt = np.clip(dt, 0.01, 10.0)  # 限制时间间隔
        else:
            dt = np.array([1.0])
        
        df['energy_J'] = power_W * dt.values
        
        return power_W
    
    def cluster_driving_behaviors(self):
        """驾驶行为聚类 - 改进版"""
        logger.info("Clustering driving behaviors with improved classification")
        
        if self.processed_data is None or self.processed_data.empty:
            logger.error("No processed data available for clustering")
            return {}, pd.DataFrame(), np.array([])
        
        data = self.processed_data.copy()
        
        # 确保所需列存在
        required_cols = self.config['clustering']['features'] + ['energy_J']
        for col in required_cols:
            if col not in data.columns:
                data[col] = 0.0
        
        print(f"\n=== 驾驶行为聚类 ===")
        for col in self.config['clustering']['features']:
            if col in data.columns:
                print(f"{col}: {data[col].min():.3f} - {data[col].max():.3f} (均值: {data[col].mean():.3f})")
        
        # K-Means聚类
        feats = self.config['clustering']['features']
        X = data[feats].fillna(0)
        
        # 数据标准化
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # PCA降维
        try:
            pca = PCA(n_components=2)
            X_pca = pca.fit_transform(X_scaled)
        except Exception as e:
            logger.warning(f"PCA failed: {e}")
            X_pca = X_scaled[:, :2] if X_scaled.shape[1] >= 2 else np.column_stack([X_scaled[:, 0], X_scaled[:, 0]])
        
        n_clusters = min(self.config['clustering']['n_clusters'], max(1, len(X) - 1))
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        clusters = kmeans.fit_predict(X_scaled)
        
        data['driver_cluster'] = clusters
        
        # 生成聚类配置文件
        cluster_profiles = {}
        for cid in range(n_clusters):
            data_c = data[data['driver_cluster'] == cid]
            if data_c.empty:
                cluster_profiles[cid] = {'size': 0, 'driver_type': 'unknown'}
                continue
            
            # 计算统计指标
            if len(data_c) > 1:
                dt = data_c['relative_time_s'].diff().fillna(1.0)
                dt = np.clip(dt, 0.01, 10.0)
                total_time = dt.sum()
                total_distance = (data_c['speed'] * dt).sum() / 1000.0  # km
            else:
                total_time = 1.0
                total_distance = 0.001
            
            total_energy = data_c['energy_J'].sum() / 3600000.0  # kWh (J->kWh)
            energy_eff = total_energy / max(total_distance, 0.001) * 100  # kWh/100km
            
            profile = {
                'size': len(data_c),
                'avg_speed': data_c['speed'].mean(),
                'std_speed': data_c['speed'].std(),
                'avg_acceleration': data_c['acceleration'].mean(),
                'std_acceleration': data_c['acceleration'].std(),
                'avg_jerk': data_c['jerk'].mean(),
                'std_jerk': data_c['jerk'].std(),
                'avg_turning_rate': data_c['turning_rate'].mean(),
                'std_turning_rate': data_c['turning_rate'].std(),
                'energy_efficiency': energy_eff,
                'total_time_s': total_time,
                'total_distance_km': total_distance,
            }
            
            # 改进的驾驶类型分类 - 适应起步阶段
            abs_avg_jerk = abs(profile['avg_jerk'])
            abs_avg_acc = abs(profile['avg_acceleration'])
            avg_speed = profile['avg_speed']
            avg_turning = profile['avg_turning_rate']
            
            # 针对起步阶段的分类逻辑
            if abs_avg_jerk > 15.0 and abs_avg_acc > 2.0:
                profile['driver_type'] = 'aggressive'
            elif avg_turning > 0.15:
                profile['driver_type'] = 'cornering'
            elif avg_speed > 8.0 and abs_avg_acc > 1.0:  # 调整速度阈值
                profile['driver_type'] = 'dynamic'
            elif avg_speed < 5.0 and abs_avg_jerk < 8.0:
                profile['driver_type'] = 'cautious'
            elif abs_avg_acc < 0.5 and abs_avg_jerk < 5.0:
                profile['driver_type'] = 'efficient'
            else:
                profile['driver_type'] = 'normal'
            
            cluster_profiles[cid] = profile
        
        print(f"聚类结果:")
        for cid, profile in cluster_profiles.items():
            print(f"  Cluster {cid} ({profile['driver_type']}): {profile['size']} 样本")
            print(f"    速度: {profile['avg_speed']*3.6:.1f} km/h, 加速度: {profile['avg_acceleration']:.2f} m/s²")
            print(f"    Jerk: {profile['avg_jerk']:.2f} m/s³, 能耗: {profile['energy_efficiency']:.1f} kWh/100km")
        
        # 更新处理后的数据
        self.processed_data = data
        
        # 在 cluster_driving_behaviors() 方法的最后，return 语句之前添加：
        print(f"DEBUG: type of self.processed_data after clustering: {type(self.processed_data)}")
        print(f"DEBUG: processed_data columns: {self.processed_data.columns.tolist() if isinstance(self.processed_data, pd.DataFrame) else 'Not DataFrame'}")
        
        return cluster_profiles, X_pca, clusters
    
    def generate_comprehensive_report(self):
        """生成综合分析报告"""
        logger.info("Generating comprehensive analysis report")
        
        if self.processed_data is None:
            logger.error("No processed data available for report generation")
            return None
        
        # 基本统计
        data = self.processed_data
        total_time = data['relative_time_s'].max()  # 使用相对时间
        total_distance = (data['speed'] * data['relative_time_s'].diff().fillna(1.0)).sum() / 1000.0  # km
        avg_speed = data['speed'].mean() * 3.6  # km/h
        max_speed = data['speed_kmh'].max()
        
        # 急刹车分析
        hard_brake_events = self.detect_hard_brake_events()
        
        # 驾驶行为聚类
        cluster_profiles, X_pca, clusters = self.cluster_driving_behaviors()

        # 确保processed_data被正确更新
        if self.processed_data is not None:
            data = self.processed_data  # 使用更新后的数据
        else:
            logger.error("processed_data is None after clustering")
            return None
        
        # 生成报告
        report = {
            'metadata': {
                'analysis_date': datetime.now().isoformat(),
                'version': VERSION,
                'data_points': len(data),
                'duration_minutes': total_time / 60,
                'total_distance_km': total_distance
            },
            'driving_summary': {
                'max_speed_kmh': max_speed,
                'avg_speed_kmh': avg_speed,
                'total_hard_brakes': len(hard_brake_events),
                'hard_brakes_per_hour': len(hard_brake_events) / max(total_time / 3600, 0.1)
            },
            'cluster_analysis': cluster_profiles,
            'recommendations': self._generate_recommendations(cluster_profiles, hard_brake_events)
        }
        
        self.results = {
            'report': report,
            'data': self.processed_data.copy(),
            'hard_brake_events': hard_brake_events,
            'cluster_profiles': cluster_profiles,
            'pca_data': X_pca,
            'clusters': clusters
        }
        
        return report
    
    def _generate_recommendations(self, cluster_profiles, hard_brake_events):
        """生成驾驶建议"""
        recommendations = []
        
        # 基于急刹车的建议
        if len(hard_brake_events) > 3:  # 降低阈值适应起步阶段
            recommendations.append({
                'category': 'Safety',
                'priority': 'High',
                'message': 'Multiple hard braking events detected during startup phase. Consider smoother acceleration and anticipating traffic conditions.',
                'impact': 'Reduces brake wear and improves passenger comfort'
            })
        elif len(hard_brake_events) == 0:
            recommendations.append({
                'category': 'Positive',
                'priority': 'Low',
                'message': 'Excellent! No hard braking events detected. Very smooth driving style.',
                'impact': 'Optimal brake system preservation and passenger comfort'
            })
        
        # 基于聚类分析的建议
        for cid, profile in cluster_profiles.items():
            if profile['driver_type'] == 'aggressive':
                recommendations.append({
                    'category': 'Efficiency',
                    'priority': 'Medium',
                    'message': f'Cluster {cid} shows aggressive driving patterns. Gentler acceleration can improve efficiency by 15-20%.',
                    'impact': f'Current energy efficiency: {profile["energy_efficiency"]:.1f} kWh/100km'
                })
            elif profile['driver_type'] == 'efficient':
                recommendations.append({
                    'category': 'Positive',
                    'priority': 'Low',
                    'message': f'Cluster {cid} demonstrates efficient driving behavior. Excellent energy management!',
                    'impact': f'Outstanding energy efficiency: {profile["energy_efficiency"]:.1f} kWh/100km'
                })
            elif profile['driver_type'] == 'cautious':
                recommendations.append({
                    'category': 'Performance',
                    'priority': 'Low',
                    'message': f'Cluster {cid} shows very cautious driving. Consider slightly more dynamic acceleration when safe.',
                    'impact': 'May improve traffic flow while maintaining safety'
                })
        
        return recommendations
    
    def visualize_results(self, save_plots=True):
        if not self.results:
            logger.error("No results available for visualization")
            return
        
        # 调试：检查数据类型
        print(f"DEBUG: self.results keys: {list(self.results.keys())}")
        print(f"DEBUG: type of self.results['data']: {type(self.results['data'])}")
        print(f"DEBUG: type of self.processed_data: {type(self.processed_data)}")
        
        # 直接使用 processed_data
        data = self.processed_data
        if data is None or not isinstance(data, pd.DataFrame):
            logger.error(f"Invalid processed_data type: {type(data)}")
            return
        
        hard_brake_events = self.results['hard_brake_events']
        cluster_profiles = self.results['cluster_profiles']
        
        # 创建综合仪表板
        fig = plt.figure(figsize=(20, 16))
        
        # 1. 速度时间曲线（使用相对时间）
        ax1 = plt.subplot(3, 3, 1)
        time_minutes = data['relative_time_s'] / 60
        
        plt.plot(time_minutes, data['speed_kmh'], 'b-', alpha=0.7, linewidth=2, label='Speed (km/h)')
        
        if not hard_brake_events.empty:
            brake_times = hard_brake_events['relative_time_s'] / 60
            plt.scatter(brake_times, hard_brake_events['speed_kmh'], 
                       color='red', marker='X', s=100, zorder=5, 
                       label=f'Hard Brakes ({len(hard_brake_events)})')
            
            # 添加急刹车标注
            for i, (idx, event) in enumerate(hard_brake_events.iterrows()):
                plt.annotate(f'Brake {i+1}', 
                           xy=(event['relative_time_s']/60, event['speed_kmh']),
                           xytext=(10, 20), textcoords='offset points',
                           bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.8),
                           arrowprops=dict(arrowstyle='->', color='red'))
        
        plt.title('Speed Profile with Hard Brake Events\n(Startup Phase Analysis)')
        plt.xlabel('Time (minutes)')
        plt.ylabel('Speed (km/h)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 2. 驾驶行为聚类PCA视图
        ax2 = plt.subplot(3, 3, 2)
        scatter = plt.scatter(self.results['pca_data'][:, 0], self.results['pca_data'][:, 1], 
                             c=self.results['clusters'], cmap='viridis', alpha=0.7, s=30)
        plt.title('Driver Behavior Clusters\n(PCA Visualization)')
        plt.xlabel('PCA Component 1')
        plt.ylabel('PCA Component 2')
        plt.colorbar(scatter, label='Cluster')
        plt.grid(True, alpha=0.3)
        
        # 3. 加速度分布
        ax3 = plt.subplot(3, 3, 3)
        colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown']
        for cluster in range(len(cluster_profiles)):
            cluster_data = data[data['driver_cluster'] == cluster]
            if not cluster_data.empty and 'acceleration' in cluster_data.columns:
                plt.hist(cluster_data['acceleration'], bins=15, alpha=0.6, 
                        label=f'Cluster {cluster}', density=True, 
                        color=colors[cluster % len(colors)])
        plt.title('Acceleration Distribution by Cluster\n(Startup Phase)')
        plt.xlabel('Acceleration (m/s²)')
        plt.ylabel('Density')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 4. Jerk分布
        ax4 = plt.subplot(3, 3, 4)
        for cluster in range(len(cluster_profiles)):
            cluster_data = data[data['driver_cluster'] == cluster]
            if not cluster_data.empty and 'jerk' in cluster_data.columns:
                plt.hist(cluster_data['jerk'], bins=15, alpha=0.6, 
                        label=f'Cluster {cluster}', density=True,
                        color=colors[cluster % len(colors)])
        plt.title('Jerk Distribution by Cluster\n(Instantaneous Values)')
        plt.xlabel('Jerk (m/s³)')
        plt.ylabel('Density')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 5. 驾驶类型分布
        ax5 = plt.subplot(3, 3, 5)
        driver_types = [cluster_profiles[cid]['driver_type'] for cid in cluster_profiles.keys()]
        type_counts = {}
        for dtype in driver_types:
            type_counts[dtype] = type_counts.get(dtype, 0) + 1
        
        colors_pie = {'aggressive': 'red', 'dynamic': 'orange', 'cornering': 'purple', 
                      'efficient': 'green', 'cautious': 'blue', 'normal': 'cyan', 'unknown': 'gray'}
        pie_colors = [colors_pie.get(dtype, 'gray') for dtype in type_counts.keys()]
        
        plt.pie(type_counts.values(), labels=type_counts.keys(), colors=pie_colors,
                autopct='%1.1f%%', startangle=90)
        plt.title('Driver Type Distribution\n(Startup Phase)')
        
        # 6. 能耗效率对比
        ax6 = plt.subplot(3, 3, 6)
        cluster_ids = list(cluster_profiles.keys())
        energy_effs = [cluster_profiles[cid]['energy_efficiency'] for cid in cluster_ids]
        driver_types_list = [cluster_profiles[cid]['driver_type'] for cid in cluster_ids]
        
        bar_colors = [colors_pie.get(dtype, 'gray') for dtype in driver_types_list]
        bars = plt.bar(range(len(cluster_ids)), energy_effs, color=bar_colors, alpha=0.8)
        plt.title('Energy Efficiency by Cluster\n(Startup Phase)')
        plt.xlabel('Cluster')
        plt.ylabel('Energy Efficiency (kWh/100km)')
        plt.xticks(range(len(cluster_ids)), [f'C{cid}\n({dtype})' for cid, dtype in zip(cluster_ids, driver_types_list)])
        
        for bar, eff in zip(bars, energy_effs):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(energy_effs)*0.01, 
                    f'{eff:.1f}', ha='center', va='bottom')
        
        # 7. 速度vs加速度散点图
        ax7 = plt.subplot(3, 3, 7)
        scatter = plt.scatter(data['speed_kmh'], data['acceleration'], 
                             c=data['driver_cluster'], cmap='viridis', alpha=0.6, s=20)
        plt.title('Speed vs Acceleration\n(Startup Phase)')
        plt.xlabel('Speed (km/h)')
        plt.ylabel('Acceleration (m/s²)')
        plt.colorbar(scatter, label='Cluster')
        plt.grid(True, alpha=0.3)
        
        # 8. 时间序列加速度
        ax8 = plt.subplot(3, 3, 8)
        sample_size = min(200, len(data))
        sample_data = data.head(sample_size)
        sample_times = sample_data['relative_time_s'] / 60
        
        plt.plot(sample_times, sample_data['acceleration'], 'g-', alpha=0.7, linewidth=1)
        plt.title(f'Acceleration Time Series\n(First {sample_size} points)')
        plt.xlabel('Time (minutes)')
        plt.ylabel('Acceleration (m/s²)')
        plt.grid(True, alpha=0.3)
        
        # 9. 综合评分仪表盘
        ax9 = plt.subplot(3, 3, 9)
        
        # 计算评分 - 适应起步阶段
        total_events = len(hard_brake_events)
        total_time_hours = data['relative_time_s'].max() / 3600
        events_per_hour = total_events / max(total_time_hours, 0.1)
        
        avg_energy_eff = np.mean([p['energy_efficiency'] for p in cluster_profiles.values()])
        
        # 评分计算（针对起步阶段调整）
        safety_score = max(0, 100 - events_per_hour * 15)  # 起步阶段降低惩罚
        efficiency_score = max(0, 100 - max(0, avg_energy_eff - 25) * 2)  # 起步阶段能耗可能较高
        smoothness_score = 100 - min(50, np.mean([abs(p['avg_jerk']) for p in cluster_profiles.values()]) * 2)
        overall_score = (safety_score + efficiency_score + smoothness_score) / 3
        
        # 绘制仪表盘
        scores = [safety_score, efficiency_score, smoothness_score, overall_score]
        labels = ['Safety', 'Efficiency', 'Smoothness', 'Overall']
        colors_gauge = ['red', 'green', 'blue', 'purple']
        
        bars = plt.bar(labels, scores, color=colors_gauge, alpha=0.7)
        plt.ylim(0, 100)
        plt.title('Eco-Driving Score Dashboard\n(Startup Phase)')
        plt.ylabel('Score (0-100)')
        
        for bar, score in zip(bars, scores):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
                    f'{score:.0f}', ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        
        if save_plots:
            plt.savefig('eco_driving_startup_analysis.png', dpi=300, bbox_inches='tight')
            logger.info("Saved startup phase analysis plot")
        
        plt.show()
        
        return True
    
    def save_results(self, output_dir='eco_driving_results'):
        """保存分析结果"""
        if not self.results:
            logger.error("No results to save")
            return False
        
        # 创建输出目录
        os.makedirs(output_dir, exist_ok=True)
        
        try:
            # 保存主要数据
            self.results['data'].to_csv(f'{output_dir}/processed_data.csv', index=False)
            
            if not self.results['hard_brake_events'].empty:
                self.results['hard_brake_events'].to_csv(f'{output_dir}/hard_brake_events.csv', index=False)
            
            # 保存JSON报告
            with open(f'{output_dir}/analysis_report.json', 'w', encoding='utf-8') as f:
                json.dump(self.results['report'], f, indent=2, ensure_ascii=False, default=str)
            
            # 保存文本报告
            self._save_text_report(f'{output_dir}/eco_driving_report.txt')
            
            logger.info(f"Results saved to {output_dir}/")
            return True
            
        except Exception as e:
            logger.error(f"Error saving results: {e}")
            return False
    
    def _save_text_report(self, filepath):
        """保存详细文本报告"""
        with open(filepath, 'w', encoding='utf-8') as f:
            report = self.results['report']
            cluster_profiles = self.results['cluster_profiles']
            hard_brake_events = self.results['hard_brake_events']
            
            f.write("=" * 80 + "\n")
            f.write("ECO-DRIVING ANALYSIS REPORT - STARTUP PHASE\n")
            f.write("=" * 80 + "\n\n")
            
            # 基本信息
            f.write(f"Analysis Date: {report['metadata']['analysis_date']}\n")
            f.write(f"System Version: {report['metadata']['version']} (Microsecond Precision)\n")
            f.write(f"Data Points: {report['metadata']['data_points']}\n")
            f.write(f"Duration: {report['metadata']['duration_minutes']:.1f} minutes\n")
            f.write(f"Total Distance: {report['metadata']['total_distance_km']:.3f} km\n\n")
            
            # 驾驶摘要
            f.write("DRIVING SUMMARY (STARTUP PHASE):\n")
            f.write("-" * 40 + "\n")
            f.write(f"Maximum Speed: {report['driving_summary']['max_speed_kmh']:.1f} km/h\n")
            f.write(f"Average Speed: {report['driving_summary']['avg_speed_kmh']:.1f} km/h\n")
            f.write(f"Total Hard Brakes: {report['driving_summary']['total_hard_brakes']}\n")
            f.write(f"Hard Brakes per Hour: {report['driving_summary']['hard_brakes_per_hour']:.1f}\n\n")
            
            # 聚类分析
            f.write("DRIVER BEHAVIOR CLUSTERS (STARTUP PHASE):\n")
            f.write("-" * 40 + "\n")
            for cid, profile in cluster_profiles.items():
                f.write(f"\nCluster {cid} - {profile['driver_type'].upper()}:\n")
                f.write(f"  Sample Size: {profile['size']} points\n")
                f.write(f"  Average Speed: {profile['avg_speed']*3.6:.1f} km/h\n")
                f.write(f"  Average Acceleration: {profile['avg_acceleration']:.3f} m/s²\n")
                f.write(f"  Average Jerk: {profile['avg_jerk']:.3f} m/s³ (instantaneous)\n")
                f.write(f"  Average Turning Rate: {profile['avg_turning_rate']:.3f} rad/s\n")
                f.write(f"  Energy Efficiency: {profile['energy_efficiency']:.2f} kWh/100km\n")
                f.write(f"  Total Distance: {profile['total_distance_km']:.4f} km\n")
                f.write(f"  Total Time: {profile['total_time_s']:.1f} seconds\n")
            
            # 急刹车详情
            f.write(f"\nHARD BRAKE EVENTS DETAIL:\n")
            f.write("-" * 40 + "\n")
            if not hard_brake_events.empty:
                for i, (idx, event) in enumerate(hard_brake_events.iterrows()):
                    time_min = event['relative_time_s'] / 60
                    f.write(f"  Event {i+1}: {time_min:.1f} min - Speed: {event['speed_kmh']:.1f} km/h, "
                           f"Acceleration: {event['acceleration']:.2f} m/s²\n")
            else:
                f.write("  No hard brake events detected - Excellent smooth driving!\n")
            
            # 建议
            f.write(f"\nRECOMMENDATIONS:\n")
            f.write("-" * 40 + "\n")
            for i, rec in enumerate(report['recommendations']):
                f.write(f"{i+1}. [{rec['category']} - {rec['priority']}] {rec['message']}\n")
                f.write(f"   Impact: {rec['impact']}\n\n")
            
            # 技术说明
            f.write("TECHNICAL NOTES:\n")
            f.write("-" * 20 + "\n")
            f.write("• Timestamp precision: Microseconds\n")
            f.write("• Analysis focus: Vehicle startup phase\n")
            f.write("• Jerk calculation: Instantaneous values preserved\n")
            f.write("• Acceleration: Real GPS-based calculation with synthetic backup\n")
            f.write("• Energy model: Adapted for low-speed urban driving\n")

# 主要使用接口
def run_complete_eco_driving_analysis(data_file, config=None):
    """
    运行完整的生态驾驶分析 - 微秒时间戳版本
    
    Args:
        data_file: 传感器数据文件路径
        config: 可选配置字典
    
    Returns:
        EcoDrivingAnalyzer实例，包含所有分析结果
    """
    print("🚗 Starting Complete Eco-Driving Analysis System v2.1")
    print("📍 Optimized for startup phase analysis with microsecond precision")
    print("=" * 70)
    
    # 初始化分析器
    analyzer = EcoDrivingAnalyzer(config)
    
    # 1. 解析数据
    print("📊 Step 1: Parsing sensor data (microsecond timestamps)...")
    if not analyzer.parse_sensor_data(data_file):
        print("❌ Failed to parse sensor data")
        return None
    
    # 2. 提取特征
    print("🔍 Step 2: Extracting driving features...")
    if not analyzer.extract_driving_features():
        print("❌ Failed to extract features")
        return None
    
    # 3. 数据同步和能耗建模
    print("⚡ Step 3: Synchronizing data and modeling energy...")
    combined_data = analyzer.synchronize_and_model_energy()
    if combined_data.empty:
        print("❌ Failed to synchronize data")
        return None
    
    # 4. 生成综合报告
    print("📋 Step 4: Generating comprehensive report...")
    report = analyzer.generate_comprehensive_report()
    if not report:
        print("❌ Failed to generate report")
        return None
    
    # 5. 可视化结果
    print("📈 Step 5: Creating visualizations...")
    analyzer.visualize_results()
    
    # 6. 保存结果
    print("💾 Step 6: Saving results...")
    analyzer.save_results()
    
    print("\n✅ Analysis Complete!")
    print("=" * 70)
    
    # 打印关键结果
    print(f"📊 Key Results (Startup Phase):")
    print(f"  • Duration: {report['metadata']['duration_minutes']:.1f} minutes")
    print(f"  • Max Speed: {report['driving_summary']['max_speed_kmh']:.1f} km/h")
    print(f"  • Hard Brake Events: {len(analyzer.results['hard_brake_events'])}")
    print(f"  • Driver Behavior Types: {len(set(p['driver_type'] for p in analyzer.results['cluster_profiles'].values()))}")
    print(f"  • Average Energy Efficiency: {np.mean([p['energy_efficiency'] for p in analyzer.results['cluster_profiles'].values()]):.1f} kWh/100km")
    
    return analyzer

# 使用示例和测试代码
if __name__ == "__main__":
    # 针对起步阶段优化的配置
    startup_config = {
        'hard_brake_threshold': -2.0,  # 降低阈值适应起步
        'speed_drop_threshold': 6,     # 降低速度下降阈值
        'min_brake_speed': 8,          # 降低最小刹车速度
        'energy_model': {
            'vehicle_mass': 1600,      # 紧凑型轿车
            'drag_coefficient': 0.26,  # 现代轿车优秀风阻
            'frontal_area': 2.2,       # 紧凑型车迎风面积
            'rolling_resistance': 0.010 # 优质轮胎
        },
        'clustering': {
            'n_clusters': 3,
            'features': ['speed', 'acceleration', 'jerk', 'turning_rate']
        }
    }
    
    # 运行分析
    data_file = "ts_1747221572.csv"  # 你的数据文件
    
    print("🚀 Running Eco-Driving Analysis - Startup Phase Optimized")
    print(f"📁 Data file: {data_file}")
    print("🔧 Configuration: Optimized for startup phase with microsecond precision")
    print("⏱️  Expected analysis time: 30-60 seconds")
    print()
    
    # 执行完整分析
    analyzer = run_complete_eco_driving_analysis(data_file, startup_config)
    
    if analyzer:
        print("\n🎉 Analysis completed successfully!")
        print("📁 Check 'eco_driving_results' folder for detailed outputs:")
        print("   • processed_data.csv - Complete processed dataset")
        print("   • hard_brake_events.csv - Hard brake event details") 
        print("   • analysis_report.json - Structured analysis results")
        print("   • eco_driving_report.txt - Human-readable report")
        print("   • eco_driving_startup_analysis.png - Comprehensive visualization")
        
        # 显示快速摘要
        print(f"\n🔍 Quick Analysis Summary:")
        report = analyzer.results['report']
        print(f"  ⏱️  Duration: {report['metadata']['duration_minutes']:.1f} minutes")
        print(f"  🚗 Max Speed: {report['driving_summary']['max_speed_kmh']:.1f} km/h")
        print(f"  🚨 Hard Brakes: {report['driving_summary']['total_hard_brakes']}")
        
        # 显示驾驶类型分布
        cluster_types = [p['driver_type'] for p in analyzer.results['cluster_profiles'].values()]
        type_counts = {}
        for dtype in cluster_types:
            type_counts[dtype] = type_counts.get(dtype, 0) + 1
        
        print(f"  🎯 Driving Types Found: {', '.join(type_counts.keys())}")
        
        # 显示主要建议
        print(f"\n💡 Top Recommendations:")
        for i, rec in enumerate(report['recommendations'][:2]):
            print(f"  {i+1}. [{rec['priority']}] {rec['message'][:80]}...")
        
        # 数据质量报告
        data = analyzer.results['data']
        print(f"\n📊 Data Quality Summary:")
        print(f"  • Total Data Points: {len(data):,}")
        print(f"  • GPS-Accelerometer Sync: {(data['sync_error_us'] < 100000).sum()}/{len(data)} points < 100ms")
        print(f"  • Acceleration Range: {data['acceleration'].min():.2f} to {data['acceleration'].max():.2f} m/s²")
        print(f"  • Jerk Range: {data['jerk'].min():.2f} to {data['jerk'].max():.2f} m/s³")
        print(f"  • Speed Range: {data['speed_kmh'].min():.1f} to {data['speed_kmh'].max():.1f} km/h")
        
        # 性能评估
        if 'hard_brake_events' in analyzer.results:
            safety_rating = "🟢 Excellent" if len(analyzer.results['hard_brake_events']) == 0 else \
                           "🟡 Good" if len(analyzer.results['hard_brake_events']) <= 2 else \
                           "🟠 Needs Improvement"
            print(f"  • Safety Rating: {safety_rating}")
        
        avg_energy = np.mean([p['energy_efficiency'] for p in analyzer.results['cluster_profiles'].values()])
        efficiency_rating = "🟢 Excellent" if avg_energy < 20 else \
                           "🟡 Good" if avg_energy < 30 else \
                           "🟠 Moderate"
        print(f"  • Efficiency Rating: {efficiency_rating} ({avg_energy:.1f} kWh/100km)")
        
        print(f"\n✨ Analysis Features Highlights:")
        print(f"  ✅ Microsecond timestamp precision")
        print(f"  ✅ Real GPS-based acceleration calculation")
        print(f"  ✅ Instantaneous jerk values (not averaged)")
        print(f"  ✅ Startup phase optimized thresholds")
        print(f"  ✅ Intelligent hard brake detection")
        print(f"  ✅ Multi-dimensional driver behavior clustering")
        print(f"  ✅ Personalized driving recommendations")
        
    else:
        print("❌ Analysis failed. Please check your data file and try again.")
        print("\n🔧 Troubleshooting Tips:")
        print("  • Ensure your CSV file exists and is readable")
        print("  • Check that the data format matches the expected structure:")
        print("    - GPS: 0,timestamp,lat,lon,alt,speed_kmh,satellites")
        print("    - Accelerometer: 1,timestamp,timestamp_us,x,y,z")
        print("    - Gyroscope: 2,timestamp,timestamp_us,x,y,z")
        print("  • Verify that timestamps are in microseconds")
        print("  • Make sure the file contains sufficient data points")

# 辅助函数：数据格式验证
def validate_data_format(file_path, sample_lines=10):
    """
    验证数据文件格式
    Args:
        file_path: 数据文件路径
        sample_lines: 检查的样本行数
    """
    print(f"🔍 Validating data format for {file_path}")
    print("=" * 50)
    
    try:
        with open(file_path, 'r') as f:
            lines = [f.readline().strip() for _ in range(sample_lines)]
        
        format_counts = {'gps': 0, 'acc': 0, 'gyro': 0, 'rot': 0, 'unknown': 0}
        
        print("Sample data lines:")
        for i, line in enumerate(lines):
            if not line:
                continue
            values = line.split(',')
            print(f"  {i+1}: {line}")
            
            if len(values) >= 2:
                try:
                    data_type = int(values[0])
                    if data_type == 0:
                        format_counts['gps'] += 1
                        if len(values) < 6:
                            print(f"    ⚠️  GPS line has only {len(values)} columns, expected 6+")
                    elif data_type == 1:
                        format_counts['acc'] += 1
                        if len(values) < 5:
                            print(f"    ⚠️  Accelerometer line has only {len(values)} columns, expected 5+")
                    elif data_type == 2:
                        format_counts['gyro'] += 1
                        if len(values) < 5:
                            print(f"    ⚠️  Gyroscope line has only {len(values)} columns, expected 5+")
                    elif data_type == 3:
                        format_counts['rot'] += 1
                        if len(values) < 6:
                            print(f"    ⚠️  Rotation line has only {len(values)} columns, expected 6+")
                    else:
                        format_counts['unknown'] += 1
                        print(f"    ⚠️  Unknown data type: {data_type}")
                except ValueError:
                    format_counts['unknown'] += 1
                    print(f"    ❌ Invalid data type: {values[0]}")
        
        print(f"\nData type distribution in sample:")
        for dtype, count in format_counts.items():
            if count > 0:
                print(f"  {dtype}: {count} lines")
        
        # 验证时间戳
        if format_counts['gps'] > 0 or format_counts['acc'] > 0:
            print(f"\nTimestamp validation:")
            for line in lines:
                if not line:
                    continue
                values = line.split(',')
                if len(values) >= 2:
                    try:
                        timestamp = int(values[1])
                        if timestamp > 1000000000000:  # 微秒级时间戳
                            print(f"  ✅ Microsecond timestamp detected: {timestamp}")
                        elif timestamp > 1000000000:   # 毫秒级时间戳
                            print(f"  ⚠️  Millisecond timestamp detected: {timestamp}")
                        else:
                            print(f"  ❌ Unusual timestamp: {timestamp}")
                        break
                    except ValueError:
                        continue
        
        return True
        
    except Exception as e:
        print(f"❌ Error validating file: {e}")
        return False

2025-05-27 15:26:56,544 - INFO - EcoDrivingAnalyzer v2.1 initialized (microsecond timestamps)
2025-05-27 15:26:56,544 - INFO - Parsing sensor data from ts_1747221572.csv


🚀 Running Eco-Driving Analysis - Startup Phase Optimized
📁 Data file: ts_1747221572.csv
🔧 Configuration: Optimized for startup phase with microsecond precision
⏱️  Expected analysis time: 30-60 seconds

🚗 Starting Complete Eco-Driving Analysis System v2.1
📍 Optimized for startup phase analysis with microsecond precision
📊 Step 1: Parsing sensor data (microsecond timestamps)...


2025-05-27 15:26:57,076 - INFO - Parsed 100000 lines from ts_1747221572.csv
2025-05-27 15:26:57,209 - INFO - Extracting driving features with microsecond precision



=== GPS数据解析 ===
GPS原始行数: 667
样本数据: ['0', '1747221671', '57.687946', '11.980719', '56.200001', '2.857636', '12']
GPS数据质量检查:
  数据点数: 667
  时间跨度: 0.0 秒 (0.0 分钟)
  速度范围: 0.0 - 91.8 km/h
  位置变化: 纬度 0.038410°, 经度 0.017737°

=== 加速度传感器数据解析 ===
加速度原始行数: 84489
样本数据: ['1', '1747221671', '3809', '-1.800781', '-0.957031', '9.539062']
加速度数据质量检查:
  数据点数: 84489
  时间跨度: 1.0 秒
  原始加速度最大值: 14.90

=== 陀螺仪数据解析 ===
  数据点数: 12051
  Z轴角速度范围: -0.529 - 0.596 rad/s
🔍 Step 2: Extracting driving features...

=== GPS特征提取 ===
GPS加速度计算结果:
  非零值: 0/667 (0.0%)
  范围: 0.000 - 0.000 m/s²
  平均: 0.000 m/s²
  标准差: 0.000 m/s²
⚠️  GPS加速度计算异常，使用改进的合成方法...
  合成加速度范围: -8.000 - 8.000 m/s²

=== 加速度传感器特征提取 ===
Magnitude统计: 4.599 - 14.913
Magnitude变化统计: -5.348178 - 5.949456
时间间隔统计: 0.000000 - 0.000221 秒


NameError: name 'jerks' is not defined